<a href="https://colab.research.google.com/github/pdeyhim/google-demo/blob/master/DatalakeDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary




# Parameters

**Before getting started lets fill in some of the variables that we need to execute some of the codes later in this notebook. **

**project_name should be set to your qwiklab project name**

**dataset_name: your dataset name. Pleas keep this name short and simple**

**table_name: your bigquery table name. Please keep this name short and simple**

In [0]:
#@markdown Forms support many types of fields.
project_name = 'deyhim-sandbox'  #@param {type: "string"}
dataset_name = 'demo3'  #@param {type: "string"}
table_name = 'random_actors2'  #@param {type: "string"}
gcs_bucket_name = 'deyhim-sandbox-demo' #@param {type: "string"}
data_source_topic_name = table_name+"_incoming_v1"
#@markdown ---


**The goal of this demo is to showcase an actual customer use-case to leverage BigQuery as the single source of truth for streaming and batch data sources. For this use-case customer’s data sources (streaming and batch) had a common schema. In order to make the pipeline simple and show the power of Dataflow, the architecture converts batch files to streaming records. The following depicts the overall architecture**

In [0]:
%%html
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:lucid="lucid" width="663.48" height="731.42"><g transform="translate(-280 -20)" lucid:page-tab-id="0_0"><path d="M502.87 302c0-1.1.9-2 2-2H698.6c1.1 0 2 .9 2 2v427.42c0 1.1-.9 2-2 2H504.88c-1.1 0-2-.9-2-2z" fill="#e0f2f1"/><use xlink:href="#a" transform="matrix(1,0,0,1,518.8695652173911,316) translate(0 12.369791666666668)"/><use xlink:href="#b" transform="matrix(1,0,0,1,518.8695652173911,316) translate(15.208333333333336 12.369791666666668)"/><use xlink:href="#c" transform="matrix(1,0,0,1,518.8695652173911,316) translate(46.11328125 12.369791666666668)"/><use xlink:href="#d" transform="matrix(1,0,0,1,518.8695652173911,316) translate(86.171875 12.369791666666668)"/><use xlink:href="#e" transform="matrix(1,0,0,1,518.8695652173911,316) translate(101.30208333333333 12.369791666666668)"/><path d="M703.48 42c0-1.1.9-2 2-2h216c1.1 0 2 .9 2 2v316c0 1.1-.9 2-2 2h-216c-1.1 0-2-.9-2-2z" fill="#f1f8e9"/><use xlink:href="#f" transform="matrix(1,0,0,1,719.4782608695654,56) translate(0 12.369791666666668)"/><use xlink:href="#g" transform="matrix(1,0,0,1,719.4782608695654,56) translate(14.830729166666668 12.369791666666668)"/><use xlink:href="#d" transform="matrix(1,0,0,1,719.4782608695654,56) translate(52.36328125 12.369791666666668)"/><use xlink:href="#h" transform="matrix(1,0,0,1,719.4782608695654,56) translate(67.49348958333333 12.369791666666668)"/><use xlink:href="#i" transform="matrix(1,0,0,1,719.4782608695654,56) translate(0 27.99479166666667)"/><path d="M300 42c0-1.1.9-2 2-2h196c1.1 0 2 .9 2 2v316c0 1.1-.9 2-2 2H302c-1.1 0-2-.9-2-2z" fill="#ffebee"/><use xlink:href="#j" transform="matrix(1,0,0,1,316,56) translate(0 12.369791666666668)"/><use xlink:href="#k" transform="matrix(1,0,0,1,316,56) translate(15.227864583333336 12.369791666666668)"/><use xlink:href="#l" transform="matrix(1,0,0,1,316,56) translate(71.62109375 12.369791666666668)"/><use xlink:href="#m" transform="matrix(1,0,0,1,316,56) translate(119.55078125 12.369791666666668)"/><use xlink:href="#n" transform="matrix(1,0,0,1,316,56) translate(0 27.99479166666667)"/><path d="M340 109.73c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v96c0 1.1-.9 2-2 2H342c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M352.5 107.73h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.58.6 1.43v2m0 8v4m0 8v4m0 8v4m0 8v4m0 8v4m0 8v4m0 8v4m0 8v2l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0H342l-1.4-.6-.6-1.4v-2m0-8v-4m0-8v-4m0-8v-4m0-8v-4m0-8v-4m0-8v-4m0-8v-4m0-8v-2l.6-1.42 1.4-.57h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M386.73 163.77l-9.63-18.4c-.4-.72-.58-1.5-.58-2.33 0-.83.2-1.6.58-2.32l9.63-18.4c.38-.73.9-1.3 1.55-1.7.65-.42 1.36-.62 2.1-.62h19.23c.76 0 1.46.22 2.12.63.65.4 1.16.97 1.55 1.68l9.63 18.42c.4.7.58 1.5.58 2.32 0 .84-.2 1.6-.58 2.33l-9.63 18.4c-.38.72-.9 1.3-1.55 1.7-.65.4-1.36.6-2.1.6h-19.27c-.75.03-1.45-.17-2.1-.58-.66-.43-1.16-1-1.52-1.73zm0 0" fill="#0e62c2"/><path d="M405.95 131.58l-5.93 1.57-6.04-1.54 3 3.33-5.05 5.8.9 1-.86 2.4 1.82 2-1.87 2.1 4.65 5.15-2.6 1.25 10.4 11.5h5.2c.75-.02 1.45-.22 2.1-.63.65-.4 1.17-.98 1.56-1.7l8-15.32z" fill="#0851b0"/><path d="M405.7 143.1c0-.8.58-1.45 1.3-1.45.74 0 1.33.65 1.33 1.45 0 .8-.6 1.46-1.32 1.46-.72 0-1.3-.66-1.3-1.46zM391.63 143.1c0-.8.6-1.45 1.32-1.45.73 0 1.32.65 1.32 1.45 0 .8-.6 1.46-1.32 1.46-.73 0-1.32-.66-1.32-1.46zM398.66 140.96c0-.8.6-1.46 1.32-1.46.73 0 1.32.65 1.32 1.46 0 .8-.6 1.45-1.32 1.45-.73 0-1.32-.64-1.32-1.44zM398.66 145.3c0-.82.6-1.47 1.32-1.47.73 0 1.32.65 1.32 1.46 0 .8-.6 1.44-1.32 1.44-.73 0-1.32-.65-1.32-1.45z" fill="#fff"/><path d="M401.96 134.16l4.03-2.58-1.95-1.22h-.08l-4 2.55-3.98-2.54h-.08l-1.94 1.22 4.02 2.58-6.07 3.84v2.72H394v-1.46l5-3.2v2.35h1.97v-2.34l4.98 3.2v1.46H408V138zM405.95 146.95l-4.98 3.23v-2.34H399v2.34l-5-3.23v-1.46h-2.07v2.7l6.07 3.88-4.02 2.55 1.94 1.25h.07l3.98-2.55 4 2.55h.07l1.94-1.25-4.04-2.55 6.04-3.88v-2.7h-2.05z" fill="#fff"/><use xlink:href="#o" transform="matrix(1,0,0,1,376.52173913043475,171.11874999999998) translate(6.384114583333332 14.557291666666668)"/><path d="M543.04 113.82c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v87.82c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M555.54 111.82h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.58.6 1.42v2.1m0 8.35v4.18m0 8.37V141m0 8.36v4.2m0 8.35v4.2m0 8.35v4.2m0 8.35v4.18m0 8.37v2.1l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.6-.6-1.4v-2.1m0-8.37V187m0-8.36v-4.2m0-8.35v-4.2m0-8.35v-4.2m0-8.35v-4.18m0-8.37v-4.18m0-8.36v-2.08l.6-1.42 1.4-.58h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M588.48 165l-9.64-16.85c-.4-.65-.58-1.36-.58-2.13 0-.77.2-1.48.58-2.13l9.6-16.9c.38-.67.9-1.2 1.55-1.58.65-.4 1.35-.58 2.1-.57h19.27c.75 0 1.46.2 2.1.58.67.4 1.18.9 1.56 1.57l9.6 16.9c.4.64.6 1.35.6 2.12s-.2 1.48-.6 2.13l-9.6 16.9c-.38.65-.9 1.17-1.55 1.56-.65.4-1.36.58-2.1.6h-19.23c-.76-.03-1.46-.24-2.1-.62-.67-.4-1.18-.92-1.57-1.58zm0 0" fill="#0e62c2"/><path d="M609.64 141l-13.27 1.22-2.57 2.6 2.2 2.22-2.16 3.9 16 16.17h1.53c.75 0 1.46-.18 2.1-.57.67-.38 1.18-.9 1.56-1.57l6.7-11.72z" fill="#0851b0"/><path d="M609.35 140.92h-15.18c-.08 0-.16.02-.23.05-.07.04-.12.1-.16.17-.04.07-.06.14-.05.22v3.16c0 .08 0 .15.05.22.04.07.1.13.16.17.07.05.15.07.23.06h15.18c.08 0 .15 0 .22-.05.07-.03.13-.1.17-.16.03-.07.05-.14.05-.22v-3.16c0-.08-.03-.15-.06-.22-.04-.07-.1-.13-.17-.17-.07-.03-.14-.06-.22-.05zM607 143.9c-.25 0-.47-.1-.65-.28-.17-.2-.27-.4-.27-.66 0-.25.1-.48.27-.66.18-.18.4-.27.65-.27.26 0 .47.1.65.27.18.18.27.4.27.66 0 .25-.1.47-.27.65-.18.2-.4.28-.65.3zm0 0M609.35 147h-15.18c-.13 0-.23.05-.3.13-.1.1-.14.2-.14.32v3.2c0 .07 0 .14.05.22.04.07.1.12.16.16.07.04.15.06.23.06h15.18c.08 0 .15-.03.22-.07.07-.04.13-.1.17-.16.03-.08.05-.15.05-.23v-3.15c0-.14-.05-.25-.14-.34-.08-.1-.2-.15-.3-.16zm-2.35 3c-.25 0-.47-.08-.65-.26-.17-.18-.27-.4-.27-.66 0-.25.1-.47.27-.65.18-.2.4-.28.65-.28.26 0 .47.1.65.28.18.18.27.4.27.65 0 .26-.1.47-.27.65-.18.2-.4.28-.65.28zm0 0" fill="#fff"/><use xlink:href="#p" transform="matrix(1,0,0,1,578.2608695652173,172.18916131422924) translate(10.664713541666666 14.557291666666668)"/><path d="M540.17 353.5c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v88.22c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M552.67 351.5h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v2.1m0 8.4v4.2m0 8.4v4.2m0 8.4v4.2m0 8.4v4.22m0 8.4v4.2m0 8.4v4.2m0 8.4v2.1l-.6 1.42-1.4.58h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.58-.6-1.42v-2.1m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4V364m0-8.4v-2.1l.6-1.43 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M585.6 412.7l-9.63-20.14c-.4-.78-.58-1.63-.58-2.55 0-.9.18-1.76.57-2.54l9.6-20.2c.38-.8.9-1.43 1.55-1.9.66-.45 1.36-.68 2.12-.67h19.26c.75 0 1.46.22 2.1.68.67.46 1.18 1.1 1.56 1.88l9.6 20.2c.4.78.6 1.63.6 2.55 0 .93-.2 1.78-.6 2.56l-9.6 20.2c-.38.8-.9 1.4-1.55 1.88-.64.46-1.35.68-2.1.7h-19.22c-.75-.04-1.46-.28-2.1-.75-.66-.47-1.18-1.1-1.57-1.9zm0 0" fill="#0e62c2"/><path d="M608.13 382.75l-2.13.58-5.35-6.48-1.72 3.86-.92 4.5 1.58 1.9-3 .76-4.18-5.02-2.83 3.28 5.17 6.26-5.5 5.28 14.56 17.62h4.7c.75 0 1.46-.23 2.1-.7.67-.45 1.18-1.07 1.56-1.87l7.56-15.88z" fill="#0851b0"/><path d="M605.1 385.66c-.54-1.12-.22-2.56.7-3.2.93-.66 2.12-.27 2.66.85.53 1.14.22 2.58-.7 3.22-.94.65-2.13.27-2.67-.86zM590.03 386.5c-.93-.64-1.24-2.08-.7-3.2.53-1.13 1.72-1.5 2.65-.86.93.65 1.24 2.08.7 3.2-.53 1.13-1.72 1.52-2.65.87zM596.94 401c0-1.3.87-2.36 1.95-2.36 1.06 0 1.93 1.06 1.93 2.36 0 1.3-.87 2.35-1.94 2.35-1.1 0-1.96-1.06-1.96-2.35zM598.44 390.94l.9-1.9 7.87 5.5-.88 1.9z" fill="#fff"/><path d="M590.55 394.58l7.87-5.5.9 1.88-7.87 5.5z" fill="#fff"/><path d="M597.97 378.98h1.8v11h-1.8z" fill="#fff"/><path d="M595.73 389.98c0-2.1 1.42-3.8 3.16-3.8 1.73 0 3.14 1.7 3.14 3.8 0 2.1-1.4 3.82-3.15 3.82s-3.17-1.7-3.17-3.82zM588.84 397c-.7-1.45-.28-3.3.92-4.13 1.2-.83 2.7-.34 3.4 1.1.7 1.45.28 3.3-.9 4.13-1.2.83-2.73.34-3.42-1.1zM605.52 398.1c-1.2-.83-1.6-2.67-.9-4.1.68-1.46 2.2-1.95 3.4-1.12 1.18.84 1.6 2.68.9 4.12-.68 1.45-2.2 1.94-3.4 1.1zM596.4 378.98c0-1.67 1.1-3.02 2.5-3.02 1.36 0 2.48 1.35 2.48 3.02 0 1.66-1.12 3.02-2.5 3.02-1.37 0-2.5-1.36-2.5-3.02z" fill="#fff"/><use xlink:href="#q" transform="matrix(1,0,0,1,575.391304347826,420.3289711850527) translate(-2.238932291666668 14.557291666666668)"/><path d="M540.17 627.5c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v79.65c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M552.67 625.5h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v1.9m0 7.6v3.78m0 7.6v3.78m0 7.6v3.78m0 7.58v3.8m0 7.58v3.8m0 7.57v3.8m0 7.58v1.9l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.6-.6-1.4v-1.9m0-7.58v-3.8m0-7.58v-3.8m0-7.6v-3.78m0-7.58v-3.8m0-7.58v-3.8m0-7.58v-3.8m0-7.57v-1.9l.6-1.4 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M585.6 679.26l-9.63-17.53c-.4-.7-.58-1.43-.58-2.23s.18-1.53.57-2.22l9.6-17.57c.38-.7.9-1.24 1.55-1.64.66-.4 1.36-.6 2.12-.6h19.26c.75 0 1.46.2 2.1.6.67.4 1.18.96 1.56 1.65l9.6 17.58c.4.7.6 1.43.6 2.22 0 .8-.2 1.54-.6 2.23l-9.6 17.57c-.38.7-.9 1.24-1.55 1.64-.64.4-1.35.6-2.1.6h-19.22c-.75-.02-1.46-.23-2.1-.64-.66-.4-1.18-.95-1.57-1.64zm0 0" fill="#0e62c2"/><path d="M604.97 653.27l-6.08-1.74-5.48 2.16-2.12 5.78 1.7 6.46 14.73 15.52h.77c.75 0 1.46-.2 2.1-.6.67-.4 1.18-.94 1.56-1.64l6.42-11.62z" fill="#0851b0"/><path d="M598.9 650.52c-2 0-3.77.65-5.32 1.96-1.55 1.3-2.54 2.97-2.98 5-.45 2.05-.23 4 .63 5.9.85 1.87 2.18 3.27 3.97 4.18 1.8.9 3.65 1.13 5.58.66 1.94-.47 3.52-1.5 4.76-3.15 1.24-1.63 1.86-3.5 1.86-5.6 0-2.46-.83-4.57-2.5-6.32-1.66-1.75-3.66-2.63-6-2.63zm0 15.76c-1.52 0-2.86-.5-4.04-1.48-1.17-1-1.93-2.25-2.26-3.8-.34-1.55-.18-3.04.48-4.47.65-1.42 1.65-2.48 3-3.17 1.37-.7 2.78-.86 4.25-.5 1.46.35 2.66 1.15 3.6 2.4.94 1.23 1.4 2.64 1.4 4.22 0 1.88-.62 3.48-1.88 4.8-1.26 1.33-2.78 2-4.56 2z" fill="#fff"/><path d="M594.85 659.14v2.78c.4.7.94 1.3 1.62 1.74v-4.52zm0 0M598.04 656.78v7.5c.54.1 1.08.1 1.62 0v-7.5zm0 0M601.38 660.3v3.32c.67-.47 1.2-1.08 1.62-1.82v-1.5zm0 0M605 665l-.87.94c-.07.08-.1.18-.1.3 0 .1.03.2.1.27l3.34 3.53c.07.08.17.12.27.12s.2-.04.28-.12l.88-.93c.07-.08.1-.17.1-.28 0-.12-.03-.22-.1-.3l-3.34-3.5c-.07-.08-.16-.13-.27-.13-.12 0-.2.04-.3.1z" fill="#fff"/><use xlink:href="#r" transform="matrix(1,0,0,1,575.3913043478261,686.5417532467532) translate(-3.2675781250000036 14.557291666666668)"/><path d="M458.4 160.66h41.82v-.24h26.5" stroke="#000" stroke-width=".5" fill="none"/><path d="M458.4 160.9h-1v-.5h1z"/><path d="M541.23 160.42l-14.26 4.64v-9.27z" stroke="#000" stroke-width=".5"/><use xlink:href="#s" transform="matrix(1,0,0,1,478.2714576338335,145.7582845704718) translate(0 7.111111111111112)"/><use xlink:href="#t" transform="matrix(1,0,0,1,478.2714576338335,145.7582845704718) translate(25.185185185185187 7.111111111111112)"/><path d="M340 228.42c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v101c0 1.1-.9 2-2 2H342c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M352.5 226.42h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v2.1m0 8.42v4.2m0 8.43v4.2m0 8.42v4.2m0 8.42v4.2m0 8.42v4.2m0 8.43v4.2m0 8.42v4.2m0 8.42v2.1l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0H342l-1.4-.6-.6-1.4v-2.1m0-8.42v-4.2m0-8.43v-4.2m0-8.42v-4.2m0-8.43v-4.2m0-8.42v-4.2m0-8.43v-4.2m0-8.42v-4.2m0-8.43v-2.1l.6-1.4 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M385.43 285.5l-9.64-19.32c-.4-.75-.58-1.57-.58-2.44 0-.88.2-1.7.57-2.44l9.63-19.33c.38-.75.9-1.35 1.55-1.78.65-.44 1.35-.65 2.1-.65h19.23c.76 0 1.46.22 2.1.65.66.44 1.18 1.03 1.57 1.77l9.63 19.33c.38.75.57 1.56.57 2.44 0 .87-.2 1.7-.57 2.44l-9.63 19.33c-.38.76-.9 1.36-1.55 1.8-.66.42-1.36.63-2.1.63h-19.27c-.76.02-1.45-.2-2.1-.62-.66-.43-1.16-1.03-1.52-1.8zm0 0" fill="#0e62c2"/><path d="M404.65 251.7l-5.94 1.66-6.03-1.62 3 3.5-5.05 6.06.92 1.06-.88 2.5 1.83 2.13-1.88 2.2 4.65 5.4-2.6 1.3 10.4 12.07h5.2c.75 0 1.45-.22 2.1-.65.66-.43 1.18-1.03 1.56-1.77l8.02-16.1z" fill="#0851b0"/><path d="M404.4 263.8c0-.84.58-1.52 1.3-1.52.73 0 1.33.68 1.33 1.52s-.6 1.53-1.32 1.53c-.72 0-1.3-.7-1.3-1.53zM390.33 263.8c0-.84.6-1.52 1.32-1.52.73 0 1.32.68 1.32 1.52s-.6 1.53-1.32 1.53c-.73 0-1.32-.7-1.32-1.53zM397.36 261.55c0-.84.6-1.53 1.32-1.53.72 0 1.32.7 1.32 1.53 0 .84-.6 1.53-1.32 1.53-.73 0-1.32-.7-1.32-1.53zM397.36 266.1c0-.85.6-1.53 1.32-1.53.72 0 1.32.68 1.32 1.53 0 .84-.6 1.53-1.32 1.53-.73 0-1.32-.7-1.32-1.53z" fill="#fff"/><path d="M400.65 254.42l4.03-2.72-1.94-1.28h-.07l-4 2.68-3.98-2.68h-.08l-1.95 1.28 4.03 2.72-6.08 4.03v2.85h2.1v-1.53l4.97-3.36v2.48h1.97v-2.47l4.98 3.37v1.53h2.05v-2.85zM404.65 267.84l-4.98 3.4v-2.47h-1.98v2.46l-5-3.4v-1.52h-2.08v2.85l6.08 4.08-4.03 2.68 1.95 1.32h.07l3.98-2.67 4 2.67h.06l1.94-1.3-4.03-2.7 6.05-4.07v-2.84h-2.05z" fill="#fff"/><use xlink:href="#u" transform="matrix(1,0,0,1,375.21739130434776,292.97014204545457) translate(-6.864583333333339 14.557291666666668)"/><path d="M398.7 332.42v65.27h125.15" stroke="#000" stroke-width=".5" fill="none"/><path d="M398.95 332.42h-.5v-1h.5z"/><path d="M538.36 397.7l-14.26 4.63v-9.27z" stroke="#000" stroke-width=".5"/><path d="M760 117.9c0-1.1.9-2 2-2h89.9c1.12 0 2 .9 2 2v79.65c0 1.1-.88 2-2 2H762c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M772.7 115.9h4.3m8.55 0h4.28m8.56 0h4.27m8.57 0h4.28m8.56 0h4.28m8.57 0h4.28m8.57 0h2.14l1.43.6.58 1.4v1.9m0 7.6v3.78m0 7.6v3.78m0 7.58v3.8m0 7.58v3.8m0 7.58v3.8m0 7.57v3.8m0 7.58v1.9l-.57 1.4-1.42.6h-2.13m-8.56 0h-4.27m-8.57 0h-4.28m-8.56 0h-4.28m-8.57 0h-4.28m-8.57 0h-4.28m-8.56 0h-4.3m-8.56 0H762l-1.4-.6-.6-1.4v-1.9m0-7.6v-3.78m0-7.58v-3.8m0-7.6v-3.78m0-7.58v-3.8m0-7.58v-3.8m0-7.58v-3.8m0-7.57v-1.9l.6-1.4 1.4-.6h2.14" stroke="#000" stroke-width="2" fill="none"/><path d="M796.34 162.97l-7.7-15.4c-.3-.6-.47-1.24-.47-1.94s.16-1.34.47-1.94l7.7-15.4c.3-.6.72-1.08 1.24-1.42.52-.35 1.1-.52 1.7-.5h15.37c.6 0 1.16.16 1.68.5.52.35.93.82 1.24 1.42l7.7 15.4c.32.6.47 1.24.47 1.93 0 .7-.15 1.35-.46 1.95l-7.7 15.4c-.3.6-.73 1.07-1.25 1.4-.52.36-1.08.52-1.68.52h-15.4c-.6.02-1.17-.15-1.7-.5-.52-.34-.92-.82-1.2-1.43zm0 0" fill="#0e62c2"/><path d="M811.72 136.05l-4.75 1.3-4.83-1.27 2.4 2.77-4.04 4.84.73.83-.7 2 1.46 1.7-1.5 1.75 3.72 4.3-2.08 1.05 8.32 9.6h4.16c.6 0 1.16-.17 1.68-.52.52-.33.94-.8 1.25-1.4l6.4-12.8z" fill="#0851b0"/><path d="M811.5 145.68c0-.67.48-1.2 1.07-1.2.58 0 1.05.53 1.05 1.2 0 .68-.47 1.22-1.05 1.22-.6 0-1.06-.54-1.06-1.22zM800.26 145.68c0-.67.48-1.2 1.06-1.2.58 0 1.05.53 1.05 1.2 0 .68-.47 1.22-1.05 1.22-.58 0-1.06-.54-1.06-1.22zM805.9 143.9c0-.68.46-1.22 1.04-1.22.58 0 1.06.54 1.06 1.2 0 .68-.48 1.23-1.06 1.23-.58 0-1.05-.54-1.05-1.2zM805.9 147.5c0-.66.46-1.2 1.04-1.2.58 0 1.06.54 1.06 1.2 0 .68-.48 1.23-1.06 1.23-.58 0-1.05-.55-1.05-1.22z" fill="#fff"/><path d="M808.52 138.2l3.23-2.15-1.56-1.02h-.07l-3.2 2.13-3.18-2.13h-.06l-1.56 1.02 3.22 2.16-4.86 3.22v2.27h1.67v-1.23l3.98-2.67v1.96h1.58v-1.96l4 2.67v1.22h1.63v-2.28zM811.72 148.9l-4 2.7v-1.95h-1.57v1.96l-3.98-2.7v-1.22h-1.67v2.27l4.86 3.24-3.22 2.13 1.55 1.05h.05l3.2-2.14 3.18 2.14h.06l1.55-1.05-3.23-2.14 4.84-3.25v-2.27h-1.64z" fill="#fff"/><use xlink:href="#v" transform="matrix(1,0,0,1,788.1739130434781,169.9356818181818) translate(-7.546223958333336 14.557291666666668)"/><path d="M743.68 156.67h-5.18V160h-78.07" stroke="#000" stroke-width=".5" fill="none"/><path d="M758.2 156.67l-14.27 4.64v-9.27z" stroke="#000" stroke-width=".5"/><use xlink:href="#w" transform="matrix(1,0,0,1,681.291959755874,145.32164049738304) translate(0 7.111111111111112)"/><use xlink:href="#x" transform="matrix(1,0,0,1,681.291959755874,145.32164049738304) translate(22.71604938271605 7.111111111111112)"/><use xlink:href="#y" transform="matrix(1,0,0,1,681.291959755874,145.32164049738304) translate(42.91358024691358 7.111111111111112)"/><path d="M540.17 491.6c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v79.64c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M552.67 489.6h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v1.9m0 7.58v3.8m0 7.58v3.8m0 7.58v3.8m0 7.57v3.8m0 7.6v3.78m0 7.6v3.78m0 7.58v1.9l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.6-.6-1.4v-1.9m0-7.58v-3.8m0-7.58v-3.8m0-7.58v-3.8m0-7.57v-3.8m0-7.58v-3.8m0-7.57v-3.8m0-7.58v-1.9l.6-1.4 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M585.6 536.67l-9.63-15.4c-.4-.6-.58-1.24-.58-1.94s.18-1.35.57-1.95l9.63-15.4c.38-.6.9-1.07 1.55-1.4.65-.35 1.36-.52 2.1-.52h19.23c.75 0 1.45.18 2.1.52.66.34 1.17.82 1.56 1.4l9.63 15.4c.4.6.58 1.25.58 1.95s-.2 1.34-.58 1.94l-9.63 15.4c-.38.6-.9 1.07-1.55 1.4-.66.36-1.37.53-2.12.52h-19.26c-.75 0-1.45-.15-2.1-.5-.66-.35-1.16-.83-1.52-1.43zm0 0" fill="#0e62c2"/><path d="M604.82 509.74l-5.93 1.32-6.05-1.3 3 2.8-5.05 4.82.9.85-.86 2 1.82 1.7-1.86 1.74 4.65 4.3-2.6 1.05 10.4 9.6h5.2c.75 0 1.44-.17 2.1-.5.65-.36 1.17-.83 1.56-1.42l8.02-12.82z" fill="#0851b0"/><path d="M604.56 519.38c0-.67.6-1.22 1.32-1.22.73 0 1.32.55 1.32 1.22 0 .67-.6 1.22-1.32 1.22-.73 0-1.32-.55-1.32-1.22zM590.5 519.38c0-.67.6-1.22 1.32-1.22.73 0 1.32.55 1.32 1.22 0 .67-.6 1.22-1.32 1.22-.73 0-1.32-.55-1.32-1.22zM597.53 517.58c0-.67.6-1.2 1.32-1.2.73 0 1.32.53 1.32 1.2 0 .68-.6 1.22-1.32 1.22-.73 0-1.32-.54-1.32-1.22zM597.53 521.2c0-.67.6-1.2 1.32-1.2.73 0 1.32.53 1.32 1.2 0 .68-.6 1.22-1.32 1.22-.73 0-1.32-.54-1.32-1.2z" fill="#fff"/><path d="M600.83 511.9l4.03-2.16-1.94-1.02h-.08l-4 2.13-3.98-2.13h-.07l-1.95 1.02 4.02 2.16-6.07 3.22v2.26h2.08v-1.22l4.98-2.67v1.95h1.98v-1.96l4.98 2.66v1.22h2.05v-2.26zM604.82 522.6l-4.98 2.7v-1.96h-1.98v1.96l-4.98-2.7v-1.22h-2.08v2.26l6.07 3.25-4.02 2.12 1.94 1.05h.06l4-2.13 3.98 2.13h.08l1.94-1.05-4.03-2.13 6.04-3.26v-2.26h-2.05z" fill="#fff"/><g><use xlink:href="#v" transform="matrix(1,0,0,1,575.3913043478261,543.6288636363637) translate(-3.0462239583333357 14.557291666666668)"/></g><path d="M598.08 574.24v34.94" stroke="#000" stroke-width=".5" fill="none"/><path d="M598.33 574.24h-.5v-1h.5z"/><path d="M598.08 623.7l-4.64-14.26h9.27z" stroke="#000" stroke-width=".5"/><path d="M595.56 444.72v28.56" stroke="#000" stroke-width=".5" fill="none"/><path d="M595.8 444.72h-.5v-1h.5z"/><path d="M595.56 487.8l-4.63-14.27h9.27z" stroke="#000" stroke-width=".5"/><path d="M805.7 200.55v195.9H673.9" stroke="#000" stroke-width=".5" fill="none"/><path d="M805.94 200.55h-.5v-1h.5z"/><path d="M659.37 396.45l14.27-4.63v9.27z" stroke="#000" stroke-width=".5"/><g><use xlink:href="#z" transform="matrix(1,0,0,1,716.798897710942,381.7862395682194) translate(0 7.111111111111112)"/><use xlink:href="#A" transform="matrix(1,0,0,1,716.798897710942,381.7862395682194) translate(31.55555555555556 7.111111111111112)"/></g><path d="M422.17 379.67h74.6v20.67h-74.6z" fill="#fff" fill-opacity="0"/><g><use xlink:href="#z" transform="matrix(1,0,0,1,427.1739130434782,384.67084381763465) translate(0 7.111111111111112)"/><use xlink:href="#A" transform="matrix(1,0,0,1,427.1739130434782,384.67084381763465) translate(31.55555555555556 7.111111111111112)"/></g><defs><path d="M688-137c238 0 323-102 359-325h193c-32 297-228 482-552 482-361 0-569-299-569-680 0-197 8-360 73-497 88-186 257-317 513-319 316-2 509 195 535 488h-193c-31-212-118-330-342-330-272 0-397 224-393 523v141c-3 282 115 517 376 517" id="B"/><path d="M248 11c-71 0-114-43-114-108 0-67 42-112 114-112s116 45 116 112c-1 65-45 108-116 108zm1-883c-71 0-114-43-114-108 0-67 42-112 114-112s116 45 116 112c-1 65-45 108-116 108" id="C"/><g id="a"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#B"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.678385416666668,0)" xlink:href="#C"/></g><path d="M1222-777c17 480-206 763-650 777H169v-1456h411c408 9 627 269 642 679zm-190 4c3-312-149-522-447-525H361v1141h202c308 0 473-209 469-531v-85" id="D"/><path d="M561-1102c238-4 403 126 403 351v498c0 99 13 178 38 237V0H808c-11-21-19-59-26-114C696-25 593 20 474 20c-199 0-368-130-365-320 4-251 214-359 490-356h180v-85c-1-135-86-212-229-212-115 0-232 67-233 171H131c20-205 206-316 430-320zM294-326c0 117 90 185 207 185 122 0 239-75 278-162v-222H634c-227 0-340 66-340 199" id="E"/><path d="M456 20C285 20 205-92 206-268v-671H9v-143h197v-262h185v262h202v143H391v671c-9 125 92 149 207 118V0c-49 13-96 20-142 20" id="F"/><g id="b"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#D"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#E"/></g><path d="M375 0H183v-1456h192V0" id="G"/><path d="M589-945c-131 0-219 81-264 174V0H140v-1082h175l6 136c83-104 191-156 324-156 229 0 346 129 346 387V0H806v-716c-2-153-65-229-217-229" id="H"/><path d="M521 20C240 20 96-239 96-550c0-303 147-552 427-552 132 0 235 47 309 140l9-120h169V-26c4 277-183 454-459 452-165-1-334-81-400-187l96-111c79 98 176 147 291 147 180 1 288-109 286-290v-93C750-23 649 20 521 20zm48-965c-209 0-287 178-287 416 0 217 86 392 285 392 117 0 202-53 257-159v-494c-57-103-142-155-255-155" id="I"/><path d="M599-131c141 0 220-65 285-146l113 88C906-50 770 20 589 20 281 21 93-214 93-545c0-223 93-397 233-485 74-48 154-72 240-72 300 2 449 218 445 537v77H278c0 197 129 357 321 357zm227-509c-3-180-88-310-260-310-170 0-264 140-282 310h542" id="J"/><path d="M538-131c121 0 232-49 232-156 0-50-20-88-56-117-73-60-334-92-420-143-92-55-162-110-162-238 0-190 192-317 400-317 223 0 414 129 413 338H759c0-108-110-186-227-186-119 0-215 53-215 159 0 45 18 78 53 101 76 52 331 90 416 139 98 57 169 115 169 251C955-92 760 20 538 20c-176 0-314-68-386-174-38-55-57-115-57-179h185c6 129 116 202 258 202" id="K"/><g id="c"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#G"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,3.6263020833333335,0)" xlink:href="#H"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,10.983072916666668,0)" xlink:href="#I"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,18.463541666666668,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,25.52734375,0)" xlink:href="#K"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,32.40234375,0)" xlink:href="#F"/></g><path d="M584 20C278 26 81-227 91-551c10-320 175-545 491-551 308-5 503 247 494 573-9 322-175 543-492 549zm-2-970c-208 0-305 185-305 421 0 216 106 398 307 398 211 0 307-186 307-420 0-214-109-399-309-399" id="L"/><g id="d"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,4.225260416666667,0)" xlink:href="#L"/></g><path d="M1160-420C1159-145 963 0 674 0H169v-1456h476c294-1 476 112 476 388 0 153-99 253-216 302 146 38 255 167 255 346zM678-157c171 2 289-95 289-261 0-175-95-263-286-263H361v524h317zm250-912c-2-172-104-229-283-229H361v463h290c157 3 278-84 277-234" id="M"/><path d="M341 0H156v-1082h185V0zm-91-1264c-68 0-108-42-109-105 0-62 41-107 109-107s110 44 110 107-42 105-110 105" id="N"/><path d="M1281-774c0 326-74 568-255 695l260 204-131 121L848 2C681 41 515 15 391-64 196-188 109-439 109-773c0-282 107-505 279-617 171-112 445-111 616-1 174 112 277 332 277 617zM696-143c312 0 393-279 393-632 0-307-114-536-395-536-268 0-393 232-393 523v107c-3 297 122 538 395 538" id="O"/><path d="M491 20c-241-1-355-148-355-398v-704h185v699c0 164 67 246 200 246 141 0 235-53 282-158v-787h185V0H812l-4-107C736-22 630 20 491 20" id="P"/><path d="M663-916c-163-27-288 18-338 148V0H140v-1082h180l3 125c61-97 147-145 258-145 36 0 63 5 82 14v172" id="Q"/><path d="M494-271l252-811h198L509 167C425 381 299 474 84 421V271c167 16 242-37 283-173l41-110L22-1082h202" id="R"/><g id="e"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#M"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#N"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,11.536458333333334,0)" xlink:href="#I"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,19.016927083333336,0)" xlink:href="#O"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,28.18359375,0)" xlink:href="#P"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.53385416666667,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.59765625000001,0)" xlink:href="#Q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,47.22656250000001,0)" xlink:href="#R"/></g><g id="f"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#M"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#C"/></g><path d="M277-555c0 244 77 420 297 424 127 2 249-93 255-210h175C980-127 805 20 574 20 258 20 81-222 92-562c11-319 164-533 481-540 237-5 426 165 431 392H829c-7-133-115-242-256-240-209 4-296 166-296 395" id="S"/><path d="M589-945c-131 0-219 81-264 174V0H140v-1536h185v585c82-101 189-151 320-151 229 0 346 129 346 387V0H806v-716c-2-153-65-229-217-229" id="T"/><g id="g"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#M"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.553385416666668,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,19.908854166666668,0)" xlink:href="#S"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,26.888020833333336,0)" xlink:href="#T"/></g><path d="M631-137c179 0 314-70 314-231 0-71-25-127-75-166s-141-77-272-115c-165-47-285-106-360-175s-112-154-112-255c0-245 231-397 495-397 206 0 369 89 449 218 43 68 65 142 65 223H942c0-184-129-283-321-283-172 0-299 76-301 236 0 59 26 109 76 150s135 78 256 113c205 59 345 116 436 256 35 53 50 119 50 193-2 256-231 398-507 390C333 12 90-129 80-422h193c0 187 160 285 358 285" id="U"/><path d="M1240-945c-141 0-250 95-250 227V0H804v-709c0-157-77-236-231-236-121 0-204 52-249 155V0H139v-1082h175l5 120c79-93 186-140 321-140 151 0 254 58 309 174 69-98 181-174 345-174 237 0 362 126 362 377V0h-185v-714c-2-159-67-231-231-231" id="V"/><g id="h"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#U"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,7.91015625,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,12.265625,0)" xlink:href="#Q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,16.653645833333336,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.71744791666667,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,30.97005208333334,0)" xlink:href="#V"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.65625000000001,0)" xlink:href="#N"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,45.89192708333334,0)" xlink:href="#H"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,53.24869791666668,0)" xlink:href="#I"/></g><path d="M497-251l268-831h189L566 0H425L33-1082h189" id="W"/><g id="i"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#B"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.678385416666668,0)" xlink:href="#L"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,16.282552083333336,0)" xlink:href="#H"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.63932291666667,0)" xlink:href="#W"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,30.013020833333336,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,37.07682291666667,0)" xlink:href="#Q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,41.58854166666667,0)" xlink:href="#K"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,48.46354166666667,0)" xlink:href="#N"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,51.69921875000001,0)" xlink:href="#L"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,59.30338541666667,0)" xlink:href="#H"/></g><path d="M973-380H363L226 0H28l556-1456h168L1309 0h-197zM421-538h495l-248-681" id="X"/><g id="j"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#X"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.697916666666668,0)" xlink:href="#C"/></g><path d="M704-1390c-164-38-297 28-288 194v114h231v143H416V0H231v-939H60v-143h171v-111c-12-274 208-416 483-347" id="Y"/><path d="M341 0H156v-1536h185V0" id="Z"/><path d="M1098-255l208-827h185L1176 0h-150L763-820 507 0H357L43-1082h184l213 810 252-810h149" id="aa"/><g id="k"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#D"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,27.604166666666668,0)" xlink:href="#Y"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,32.23307291666667,0)" xlink:href="#Z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.46875000000001,0)" xlink:href="#L"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.07291666666667,0)" xlink:href="#aa"/></g><path d="M632-1102c291 0 422 251 422 573 0 297-141 548-419 549-131 0-235-42-310-125v521H140v-1498h169l9 120c75-93 180-140 314-140zm-53 971c207 0 290-180 290-419 0-218-92-395-292-395-112 0-196 50-252 149v517c55 99 140 148 254 148" id="ab"/><g id="l"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#U"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,7.91015625,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.162760416666668,0)" xlink:href="#V"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,26.848958333333336,0)" xlink:href="#ab"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,34.329427083333336,0)" xlink:href="#Z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,37.56510416666667,0)" xlink:href="#J"/></g><g id="m"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#D"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#E"/></g><path d="M730-137c145 0 253-29 323-105v-327H716v-156h528v534C1148-51 964 20 729 20c-394 0-598-279-607-678v-127c-5-410 194-690 585-691 310-1 499 171 537 447h-192c-36-193-151-289-344-289-285 0-393 220-393 528v119c-5 296 140 534 415 534" id="ac"/><g id="n"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#ac"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,9.08203125,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,16.145833333333336,0)" xlink:href="#H"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.50260416666667,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,30.566406250000007,0)" xlink:href="#Q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,34.81770833333334,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.07031250000001,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,46.29557291666667,0)" xlink:href="#L"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,53.899739583333336,0)" xlink:href="#Q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,58.411458333333336,0)" xlink:href="#K"/></g><g id="o"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#M"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.553385416666668,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,19.908854166666668,0)" xlink:href="#S"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,26.888020833333336,0)" xlink:href="#T"/></g><g id="p"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#ac"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,9.08203125,0)" xlink:href="#B"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,17.760416666666668,0)" xlink:href="#U"/></g><path d="M1216-1011c-1 297-202 441-512 441H361V0H169v-1456h537c297-6 510 164 510 445zm-192 2c1-178-126-288-307-289H361v571h345c196 1 318-92 318-282" id="ad"/><path d="M634-1102c292 0 422 254 422 573 0 298-143 547-420 549-137 0-242-48-317-145L310 0H140v-1536h185v573c75-93 178-139 309-139zm-44 965c207 0 281-175 281-413 0-224-79-395-283-395-122 0-210 57-263 170v468c57 113 145 170 265 170" id="ae"/><path d="M177 125H18l608-1581h158" id="af"/><g id="q"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#ad"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.411458333333334,0)" xlink:href="#P"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.76171875,0)" xlink:href="#ae"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.2421875,0)" xlink:href="#af"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,28.736979166666668,0)" xlink:href="#U"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,36.64713541666667,0)" xlink:href="#P"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.997395833333336,0)" xlink:href="#ae"/></g><g id="r"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#M"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#N"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,11.536458333333334,0)" xlink:href="#I"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,19.016927083333336,0)" xlink:href="#O"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,28.18359375,0)" xlink:href="#P"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.53385416666667,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.59765625000001,0)" xlink:href="#Q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,47.22656250000001,0)" xlink:href="#R"/></g><path d="M160-131c35 5 61 23 61 61C221 17 115-2 30 0v-248c76 3 177-17 177 60 0 33-19 50-47 57zm-97-11c50-1 110 9 110-42 0-47-63-36-110-37v79zm0 115c55-2 124 14 124-45 0-56-70-42-124-44v89" id="ag"/><path d="M141-36C126-15 110 5 73 4 37 3 15-17 15-53c-1-64 63-63 125-63 3-35-9-54-41-54-24 1-41 7-42 31l-33-3c5-37 33-52 76-52 45 0 72 20 72 64v82c-1 20 7 32 28 27v20c-31 9-61-2-59-35zM48-53c0 20 12 33 32 33 41-3 63-29 60-74-43 2-92-5-92 41" id="ah"/><path d="M59-47c-2 24 18 29 38 22v24C64 9 27 4 27-40v-127H5v-23h24l9-43h21v43h35v23H59v120" id="ai"/><path d="M96-169c-40 0-48 33-48 73s9 75 48 75c24 0 41-14 43-38l32 2c-6 37-31 61-74 61-59 0-76-41-82-99-10-93 101-131 147-64 4 7 5 14 7 22l-32 3c-4-21-16-35-41-35" id="aj"/><path d="M106-169C34-169 62-67 57 0H25v-261h32l-1 103c12-21 28-36 61-36 89 0 53 116 60 194h-32v-121c2-32-8-49-39-48" id="ak"/><g id="s"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#ag"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.9259259259259265,0)" xlink:href="#ah"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,10.8641975308642,0)" xlink:href="#ai"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,13.333333333333334,0)" xlink:href="#aj"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,17.77777777777778,0)" xlink:href="#ak"/></g><path d="M63-220v92h138v28H63V0H30v-248h175v28H63" id="al"/><path d="M24-231v-30h32v30H24zM24 0v-190h32V0H24" id="am"/><path d="M24 0v-261h32V0H24" id="an"/><path d="M100-194c63 0 86 42 84 106H49c0 40 14 67 53 68 26 1 43-12 49-29l28 8c-11 28-37 45-77 45C44 4 14-33 15-96c1-61 26-98 85-98zm52 81c6-60-76-77-97-28-3 7-6 17-6 28h103" id="ao"/><path d="M135-143c-3-34-86-38-87 0 15 53 115 12 119 90S17 21 10-45l28-5c4 36 97 45 98 0-10-56-113-15-118-90-4-57 82-63 122-42 12 7 21 19 24 35" id="ap"/><g id="t"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#al"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.407407407407407,0)" xlink:href="#am"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,7.358024691358025,0)" xlink:href="#an"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,9.308641975308642,0)" xlink:href="#ao"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,14.246913580246915,0)" xlink:href="#ap"/></g><g id="u"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#U"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,7.91015625,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,12.265625,0)" xlink:href="#Q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,16.653645833333336,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.71744791666667,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,30.97005208333334,0)" xlink:href="#V"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.65625000000001,0)" xlink:href="#N"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,45.89192708333334,0)" xlink:href="#H"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,53.24869791666668,0)" xlink:href="#I"/></g><g id="v"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#D"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,27.604166666666668,0)" xlink:href="#Y"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,32.23307291666667,0)" xlink:href="#Z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.46875000000001,0)" xlink:href="#L"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.07291666666667,0)" xlink:href="#aa"/></g><path d="M185-189c-5-48-123-54-124 2 14 75 158 14 163 119 3 78-121 87-175 55-17-10-28-26-33-46l33-7c5 56 141 63 141-1 0-78-155-14-162-118-5-82 145-84 179-34 5 7 8 16 11 25" id="aq"/><path d="M117-194c89-4 53 116 60 194h-32v-121c0-31-8-49-39-48C34-167 62-67 57 0H25l-1-190h30c1 10-1 24 2 32 11-22 29-35 61-36" id="ar"/><g id="w"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#aq"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.9259259259259265,0)" xlink:href="#aj"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,10.37037037037037,0)" xlink:href="#ah"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,15.308641975308642,0)" xlink:href="#ar"/></g><path d="M190 0L58-211 59 0H30v-248h39L202-35l-2-213h31V0h-41" id="as"/><path d="M206 0h-36l-40-164L89 0H53L-1-190h32L70-26l43-164h34l41 164 42-164h31" id="at"/><g id="x"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#as"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,6.395061728395063,0)" xlink:href="#ao"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,11.333333333333336,0)" xlink:href="#at"/></g><g id="y"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#al"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.407407407407407,0)" xlink:href="#am"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,7.358024691358025,0)" xlink:href="#an"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,9.308641975308642,0)" xlink:href="#ao"/></g><path d="M30-248c87 1 191-15 191 75 0 78-77 80-158 76V0H30v-248zm33 125c57 0 124 11 124-50 0-59-68-47-124-48v98" id="au"/><path d="M84 4C-5 8 30-112 23-190h32v120c0 31 7 50 39 49 72-2 45-101 50-169h31l1 190h-30c-1-10 1-25-2-33-11 22-28 36-60 37" id="av"/><path d="M115-194c53 0 69 39 70 98 0 66-23 100-70 100C84 3 66-7 56-30L54 0H23l1-261h32v101c10-23 28-34 59-34zm-8 174c40 0 45-34 45-75 0-40-5-75-45-74-42 0-51 32-51 76 0 43 10 73 51 73" id="aw"/><g id="z"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#au"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.9259259259259265,0)" xlink:href="#av"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,10.8641975308642,0)" xlink:href="#aw"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,15.80246913580247,0)" xlink:href="#an"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,17.75308641975309,0)" xlink:href="#am"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,19.703703703703706,0)" xlink:href="#ap"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,24.148148148148152,0)" xlink:href="#ak"/></g><path d="M233-177c-1 41-23 64-60 70L243 0h-38l-65-103H63V0H30v-248c88 3 205-21 203 71zM63-129c60-2 137 13 137-47 0-61-80-42-137-45v92" id="ax"/><path d="M100-194c62-1 85 37 85 99 1 63-27 99-86 99S16-35 15-95c0-66 28-99 85-99zM99-20c44 1 53-31 53-75 0-43-8-75-51-75s-53 32-53 75 10 74 51 75" id="ay"/><path d="M114-163C36-179 61-72 57 0H25l-1-190h30c1 12-1 29 2 39 6-27 23-49 58-41v29" id="az"/><path d="M85-194c31 0 48 13 60 33l-1-100h32l1 261h-30c-2-10 0-23-3-31C134-8 116 4 85 4 32 4 16-35 15-94c0-66 23-100 70-100zm9 24c-40 0-46 34-46 75 0 40 6 74 45 74 42 0 51-32 51-76 0-42-9-74-50-73" id="aA"/><g id="A"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#ax"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,6.395061728395063,0)" xlink:href="#ao"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,11.333333333333336,0)" xlink:href="#aj"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,15.777777777777779,0)" xlink:href="#ay"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,20.71604938271605,0)" xlink:href="#az"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,23.654320987654323,0)" xlink:href="#aA"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,28.592592592592595,0)" xlink:href="#ap"/></g></defs></g></svg>


**A: Initially the Dataflow sample data generators will generate both streaming and batch data. Streaming dataflow job publishes records to pubsub and batch dataflow job stores batch data in GCS.**

**B: Batch to streaming conversion dataflow job will scan GCS for any new batch file generated by the batch job and converts each line in the batch file to a record that later gets published to the streaming pubsub topic. **

**C: Data ingest to BigQuery dataflow job will read data from the pubsub topic that now has both streaming and batch records and publishes the records to BigQuery.**


# Installing dependencies

**Lets begin with installing our dependencies**

In [0]:
!pip install google-cloud-pubsub

In [0]:
!apt-get install jq

In [0]:
from google.colab import auth
from google.cloud import pubsub_v1
from google.cloud import bigquery
from google.cloud import storage

import os
import subprocess

**The following cell authenticates you before we can start running the code examples. You'll be asked to copy a given code to the text box provided below.**

In [0]:
auth.authenticate_user()

**The goal of this demo is to demonstrate simplicity but yet powerful combination of dataflow and bigquery**

# Creating Google Cloud Resources

**We first begin with creating BigQuery and PubSub resources required to ingest and store data. **


**Creating PubSub Topic**

In [0]:
topic_name = 'projects/{project_id}/topics/{topic}'.format(
    project_id=project_name,
    topic=data_source_topic_name, 
)
publisher = pubsub_v1.PublisherClient()
print("Created PubSub Topic: {}".format(data_source_topic_name))

try:
  publisher.create_topic(topic_name)
except Exception as e: print(e)

**We're using BigQuery as our data lake. For this specific dataset we're creating a table that will hold raw data**

In [0]:
bq_client = bigquery.Client(project=project_name)

In [0]:
bq_client.create_dataset(dataset=dataset_name)

In [0]:
bq_client.query('''CREATE OR REPLACE TABLE `{dataset}.{table}` (ts timestamp, user_id INT64, actor STRING, location STRUCT<zip STRING, longitude	 STRING, latitude	STRING>, v1 FLOAT64, v2 FLOAT64, v3 FLOAT64, v4 FLOAT64, v5 FLOAT64, v6 FLOAT64) 
  PARTITION BY DATE(ts)'''.format(dataset=dataset_name,table=table_name)).result()

**Creating GCS Bucket that will hold some configurations and also incoming batch data**

In [0]:
storage_client = storage.Client(project=project_name)

In [0]:
storage_client = storage.Client(project=project_name)

try: 
  storage_client.create_bucket(gcs_bucket_name)
  print("bucket created")
except Exception as e: print(e)


# Generating Data


In [0]:
%%html 
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:lucid="lucid" width="423.3" height="443.72"><g transform="translate(-740.1739130434783 -87.53162270020425)" lucid:page-tab-id="nCGV3-YpBOqE"><path d="M923.48 109.53c0-1.1.9-2 2-2h216c1.1 0 2 .9 2 2v316c0 1.1-.9 2-2 2h-216c-1.1 0-2-.9-2-2z" fill="#f1f8e9"/><use xlink:href="#a" transform="matrix(1,0,0,1,939.4782608695652,123.53162270020425) translate(0 12.369791666666668)"/><use xlink:href="#b" transform="matrix(1,0,0,1,939.4782608695652,123.53162270020425) translate(14.830729166666668 12.369791666666668)"/><use xlink:href="#c" transform="matrix(1,0,0,1,939.4782608695652,123.53162270020425) translate(52.36328125 12.369791666666668)"/><use xlink:href="#d" transform="matrix(1,0,0,1,939.4782608695652,123.53162270020425) translate(67.49348958333333 12.369791666666668)"/><use xlink:href="#e" transform="matrix(1,0,0,1,939.4782608695652,123.53162270020425) translate(0 27.99479166666667)"/><path d="M763.04 181.35c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v87.82c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M775.54 179.35h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v2.1m0 8.35v4.2m0 8.35v4.18m0 8.37v4.18m0 8.36v4.18m0 8.37v4.17m0 8.36v4.18m0 8.38v2.1l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.6-.6-1.4v-2.1m0-8.37v-4.17m0-8.36V242m0-8.38v-4.18m0-8.36v-4.18m0-8.37v-4.18m0-8.36v-4.2m0-8.36v-2.1l.6-1.4 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M808.48 232.53l-9.64-16.84c-.4-.67-.58-1.38-.58-2.15 0-.76.2-1.48.58-2.13l9.6-16.88c.38-.67.9-1.2 1.55-1.6.65-.37 1.35-.56 2.1-.56h19.27c.75 0 1.46.2 2.1.58.67.4 1.18.9 1.56 1.58l9.6 16.88c.4.65.6 1.37.6 2.13 0 .77-.2 1.48-.6 2.14l-9.6 16.87c-.38.66-.9 1.2-1.55 1.57-.65.4-1.36.58-2.1.58h-19.23c-.76-.02-1.46-.22-2.1-.6-.67-.4-1.18-.93-1.57-1.6zm0 0" fill="#0e62c2"/><path d="M829.64 208.52l-13.27 1.23-2.57 2.6 2.2 2.22-2.16 3.9 16 16.17h1.53c.75 0 1.46-.2 2.1-.58.67-.38 1.18-.9 1.56-1.57l6.7-11.73z" fill="#0851b0"/><path d="M829.35 208.45h-15.18c-.08 0-.16.02-.23.05-.07.04-.12.1-.16.17-.04.07-.06.14-.05.22v3.15c0 .08 0 .15.05.22.04.08.1.13.16.17.07.04.15.06.23.05h15.18c.08 0 .15-.02.22-.06.07-.04.13-.1.17-.17.03-.07.05-.14.05-.22v-3.16c0-.1-.03-.16-.06-.23-.04-.07-.1-.13-.17-.17-.07-.03-.14-.05-.22-.05zm-2.35 2.97c-.25 0-.47-.1-.65-.27-.17-.18-.27-.4-.27-.66s.1-.48.27-.67c.18-.18.4-.27.65-.27.26 0 .47.1.65.27.18.2.27.4.27.66 0 .24-.1.46-.27.64-.18.18-.4.27-.65.28zm0 0M829.35 214.53h-15.18c-.13 0-.23.05-.3.14-.1.08-.14.2-.14.3v3.2c0 .08 0 .15.05.23.04.07.1.12.16.16.07.04.15.06.23.06h15.18c.08 0 .15-.02.22-.06.07-.04.13-.1.17-.16.03-.08.05-.15.05-.23v-3.15c0-.13-.05-.24-.14-.33-.08-.1-.2-.16-.3-.17zm-2.35 3c-.25 0-.47-.08-.65-.26-.17-.18-.27-.4-.27-.66 0-.24.1-.46.27-.64.18-.18.4-.27.65-.27.26 0 .47.08.65.26.18.18.27.4.27.65 0 .27-.1.48-.27.67-.18.18-.4.27-.65.27zm0 0" fill="#fff"/><use xlink:href="#f" transform="matrix(1,0,0,1,798.2608695652173,239.72078401443343) translate(10.664713541666666 14.557291666666668)"/><path d="M760.17 421.04c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v88.2c0 1.12-.9 2-2 2h-113.4c-1.1 0-2-.88-2-2z" fill="#fff"/><path d="M772.67 419.04h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.58.6 1.42v2.1m0 8.4v4.2m0 8.4v4.2m0 8.4v4.2m0 8.4v4.2m0 8.4v4.2m0 8.4v4.2m0 8.4v2.1l-.6 1.43-1.4.58h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.58-.6-1.42v-2.1m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-2.1l.6-1.43 1.4-.58h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M805.6 480.24l-9.63-20.15c-.4-.8-.58-1.64-.58-2.56 0-.92.18-1.77.57-2.55l9.6-20.2c.38-.8.9-1.44 1.55-1.9.66-.46 1.36-.7 2.12-.68h19.26c.75 0 1.46.23 2.1.7.67.46 1.18 1.08 1.56 1.87l9.6 20.2c.4.77.6 1.62.6 2.54 0 .92-.2 1.77-.6 2.55l-9.6 20.2c-.38.78-.9 1.4-1.55 1.87-.64.46-1.35.7-2.1.7h-19.22c-.75-.03-1.46-.28-2.1-.74-.66-.47-1.18-1.1-1.57-1.9zm0 0" fill="#0e62c2"/><path d="M828.13 450.28l-2.13.58-5.35-6.48-1.72 3.86-.92 4.48 1.58 1.9-3 .77-4.18-5.03-2.83 3.3 5.17 6.25-5.5 5.27 14.56 17.62h4.7c.75 0 1.46-.24 2.1-.7.67-.46 1.18-1.08 1.56-1.88l7.56-15.9z" fill="#0851b0"/><path d="M825.1 453.2c-.54-1.13-.22-2.57.7-3.22.93-.64 2.12-.26 2.66.86.53 1.13.22 2.57-.7 3.22-.94.64-2.13.26-2.67-.86zM810.03 454.04c-.93-.65-1.24-2.1-.7-3.2.53-1.13 1.72-1.52 2.65-.87.93.65 1.24 2.1.7 3.2-.53 1.13-1.72 1.52-2.65.87zM816.94 468.53c0-1.3.87-2.36 1.95-2.36 1.06 0 1.93 1.06 1.93 2.36 0 1.3-.87 2.35-1.94 2.35-1.1 0-1.96-1.06-1.96-2.35zM818.44 458.47l.9-1.9 7.87 5.5-.88 1.9z" fill="#fff"/><path d="M810.55 462.1l7.87-5.5.9 1.9-7.87 5.5z" fill="#fff"/><path d="M817.97 446.5h1.8v11.02h-1.8z" fill="#fff"/><path d="M815.73 457.52c0-2.1 1.42-3.82 3.16-3.82 1.73 0 3.14 1.7 3.14 3.82 0 2.1-1.4 3.8-3.15 3.8s-3.17-1.7-3.17-3.8zM808.84 464.53c-.7-1.45-.28-3.3.92-4.13 1.2-.83 2.7-.34 3.4 1.1.7 1.45.28 3.3-.9 4.13-1.2.84-2.73.34-3.42-1.1zM825.52 465.64c-1.2-.84-1.6-2.68-.9-4.12.68-1.45 2.2-1.94 3.4-1.1 1.18.83 1.6 2.67.9 4.12-.68 1.44-2.2 1.94-3.4 1.1zM816.4 446.5c0-1.66 1.1-3 2.5-3 1.36 0 2.48 1.34 2.48 3 0 1.68-1.12 3.03-2.5 3.03-1.37 0-2.5-1.35-2.5-3.02z" fill="#fff"/><use xlink:href="#g" transform="matrix(1,0,0,1,795.391304347826,487.86059388525695) translate(-2.238932291666668 14.557291666666668)"/><path d="M980 185.44c0-1.1.9-2 2-2h89.9c1.12 0 2 .9 2 2v79.64c0 1.1-.88 2-2 2H982c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M992.7 183.44h4.3m8.55 0h4.28m8.56 0h4.27m8.57 0h4.28m8.56 0h4.28m8.57 0h4.28m8.57 0h2.14l1.43.6.58 1.4v1.9m0 7.58v3.8m0 7.58v3.8m0 7.57v3.8m0 7.58v3.8m0 7.58v3.8m0 7.57v3.8m0 7.58v1.9l-.57 1.4-1.42.6h-2.13m-8.56 0h-4.27m-8.57 0h-4.28m-8.56 0h-4.28m-8.57 0h-4.28m-8.57 0h-4.28m-8.56 0h-4.3m-8.56 0H982l-1.4-.6-.6-1.4v-1.9m0-7.58v-3.8m0-7.58v-3.8m0-7.58v-3.8m0-7.57v-3.8m0-7.58v-3.8m0-7.6v-3.78m0-7.58v-1.9l.6-1.4 1.4-.6h2.14" stroke="#000" stroke-width="2" fill="none"/><path d="M1016.34 230.5l-7.7-15.4c-.3-.6-.47-1.24-.47-1.93 0-.7.16-1.35.47-1.95l7.7-15.4c.3-.6.72-1.07 1.24-1.4.52-.36 1.1-.52 1.7-.52h15.37c.6 0 1.16.18 1.68.52s.93.8 1.24 1.4l7.7 15.4c.32.6.47 1.25.47 1.95s-.15 1.34-.46 1.94l-7.7 15.4c-.3.6-.73 1.08-1.25 1.42-.52.35-1.08.52-1.68.5h-15.4c-.6.03-1.17-.13-1.7-.48-.52-.35-.92-.83-1.2-1.44zm0 0" fill="#0e62c2"/><path d="M1031.72 203.58l-4.75 1.32-4.83-1.3 2.4 2.78-4.04 4.84.73.85-.7 2 1.46 1.68-1.5 1.76 3.72 4.3-2.08 1.06 8.32 9.6h4.16c.6 0 1.16-.17 1.68-.5.52-.36.94-.83 1.25-1.42l6.4-12.82z" fill="#0851b0"/><path d="M1031.5 213.22c0-.68.48-1.22 1.07-1.22.58 0 1.05.54 1.05 1.22 0 .67-.47 1.2-1.05 1.2-.6 0-1.06-.53-1.06-1.2zM1020.26 213.22c0-.68.48-1.22 1.06-1.22.58 0 1.05.54 1.05 1.22 0 .67-.47 1.2-1.05 1.2-.58 0-1.06-.53-1.06-1.2zM1025.9 211.42c0-.67.46-1.2 1.04-1.2.58 0 1.06.53 1.06 1.2 0 .67-.48 1.22-1.06 1.22-.58 0-1.05-.55-1.05-1.22zM1025.9 215.04c0-.67.46-1.2 1.04-1.2.58 0 1.06.53 1.06 1.2 0 .68-.48 1.22-1.06 1.22-.58 0-1.05-.54-1.05-1.22z" fill="#fff"/><path d="M1028.52 205.74l3.23-2.16-1.56-1.02h-.07l-3.2 2.13-3.18-2.14h-.06l-1.56 1.02 3.22 2.16-4.86 3.2v2.28h1.67V210l3.98-2.67v1.96h1.58v-1.97l4 2.67v1.22h1.63v-2.27zM1031.72 216.43l-4 2.7v-1.95h-1.57v1.96l-3.98-2.7v-1.23h-1.67v2.28l4.86 3.25-3.22 2.13 1.55 1.05h.05l3.2-2.12 3.18 2.13h.06l1.55-1.04-3.23-2.13 4.84-3.25v-2.27h-1.64z" fill="#fff"/><use xlink:href="#h" transform="matrix(1,0,0,1,1008.1739130434781,237.46730451838613) translate(-7.546223958333336 14.557291666666668)"/><path d="M963.68 224.2h-5.18v3.32h-78.07" stroke="#000" stroke-width=".5" fill="none"/><path d="M978.2 224.2l-14.27 4.64v-9.27z" stroke="#000" stroke-width=".5"/><use xlink:href="#i" transform="matrix(1,0,0,1,901.2919597558741,212.8532631975873) translate(0 7.111111111111112)"/><use xlink:href="#j" transform="matrix(1,0,0,1,901.2919597558741,212.8532631975873) translate(22.71604938271605 7.111111111111112)"/><use xlink:href="#k" transform="matrix(1,0,0,1,901.2919597558741,212.8532631975873) translate(42.91358024691358 7.111111111111112)"/><path d="M1025.7 268.08v195.9H893.9" stroke="#000" stroke-width=".5" fill="none"/><path d="M1025.94 268.08h-.5v-1h.5z"/><path d="M879.37 463.98l14.27-4.63v9.27z" stroke="#000" stroke-width=".5"/><g><use xlink:href="#l" transform="matrix(1,0,0,1,936.7988977109419,449.31786226842365) translate(0 7.111111111111112)"/><use xlink:href="#m" transform="matrix(1,0,0,1,936.7988977109419,449.31786226842365) translate(31.55555555555556 7.111111111111112)"/></g><defs><path d="M1160-420C1159-145 963 0 674 0H169v-1456h476c294-1 476 112 476 388 0 153-99 253-216 302 146 38 255 167 255 346zM678-157c171 2 289-95 289-261 0-175-95-263-286-263H361v524h317zm250-912c-2-172-104-229-283-229H361v463h290c157 3 278-84 277-234" id="n"/><path d="M248 11c-71 0-114-43-114-108 0-67 42-112 114-112s116 45 116 112c-1 65-45 108-116 108zm1-883c-71 0-114-43-114-108 0-67 42-112 114-112s116 45 116 112c-1 65-45 108-116 108" id="o"/><g id="a"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#n"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#o"/></g><path d="M561-1102c238-4 403 126 403 351v498c0 99 13 178 38 237V0H808c-11-21-19-59-26-114C696-25 593 20 474 20c-199 0-368-130-365-320 4-251 214-359 490-356h180v-85c-1-135-86-212-229-212-115 0-232 67-233 171H131c20-205 206-316 430-320zM294-326c0 117 90 185 207 185 122 0 239-75 278-162v-222H634c-227 0-340 66-340 199" id="p"/><path d="M456 20C285 20 205-92 206-268v-671H9v-143h197v-262h185v262h202v143H391v671c-9 125 92 149 207 118V0c-49 13-96 20-142 20" id="q"/><path d="M277-555c0 244 77 420 297 424 127 2 249-93 255-210h175C980-127 805 20 574 20 258 20 81-222 92-562c11-319 164-533 481-540 237-5 426 165 431 392H829c-7-133-115-242-256-240-209 4-296 166-296 395" id="r"/><path d="M589-945c-131 0-219 81-264 174V0H140v-1536h185v585c82-101 189-151 320-151 229 0 346 129 346 387V0H806v-716c-2-153-65-229-217-229" id="s"/><g id="b"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#n"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.553385416666668,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,19.908854166666668,0)" xlink:href="#r"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,26.888020833333336,0)" xlink:href="#s"/></g><path d="M584 20C278 26 81-227 91-551c10-320 175-545 491-551 308-5 503 247 494 573-9 322-175 543-492 549zm-2-970c-208 0-305 185-305 421 0 216 106 398 307 398 211 0 307-186 307-420 0-214-109-399-309-399" id="t"/><g id="c"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,4.225260416666667,0)" xlink:href="#t"/></g><path d="M631-137c179 0 314-70 314-231 0-71-25-127-75-166s-141-77-272-115c-165-47-285-106-360-175s-112-154-112-255c0-245 231-397 495-397 206 0 369 89 449 218 43 68 65 142 65 223H942c0-184-129-283-321-283-172 0-299 76-301 236 0 59 26 109 76 150s135 78 256 113c205 59 345 116 436 256 35 53 50 119 50 193-2 256-231 398-507 390C333 12 90-129 80-422h193c0 187 160 285 358 285" id="u"/><path d="M663-916c-163-27-288 18-338 148V0H140v-1082h180l3 125c61-97 147-145 258-145 36 0 63 5 82 14v172" id="v"/><path d="M599-131c141 0 220-65 285-146l113 88C906-50 770 20 589 20 281 21 93-214 93-545c0-223 93-397 233-485 74-48 154-72 240-72 300 2 449 218 445 537v77H278c0 197 129 357 321 357zm227-509c-3-180-88-310-260-310-170 0-264 140-282 310h542" id="w"/><path d="M1240-945c-141 0-250 95-250 227V0H804v-709c0-157-77-236-231-236-121 0-204 52-249 155V0H139v-1082h175l5 120c79-93 186-140 321-140 151 0 254 58 309 174 69-98 181-174 345-174 237 0 362 126 362 377V0h-185v-714c-2-159-67-231-231-231" id="x"/><path d="M341 0H156v-1082h185V0zm-91-1264c-68 0-108-42-109-105 0-62 41-107 109-107s110 44 110 107-42 105-110 105" id="y"/><path d="M589-945c-131 0-219 81-264 174V0H140v-1082h175l6 136c83-104 191-156 324-156 229 0 346 129 346 387V0H806v-716c-2-153-65-229-217-229" id="z"/><path d="M521 20C240 20 96-239 96-550c0-303 147-552 427-552 132 0 235 47 309 140l9-120h169V-26c4 277-183 454-459 452-165-1-334-81-400-187l96-111c79 98 176 147 291 147 180 1 288-109 286-290v-93C750-23 649 20 521 20zm48-965c-209 0-287 178-287 416 0 217 86 392 285 392 117 0 202-53 257-159v-494c-57-103-142-155-255-155" id="A"/><g id="d"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#u"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,7.91015625,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,12.265625,0)" xlink:href="#v"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,16.653645833333336,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.71744791666667,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,30.97005208333334,0)" xlink:href="#x"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.65625000000001,0)" xlink:href="#y"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,45.89192708333334,0)" xlink:href="#z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,53.24869791666668,0)" xlink:href="#A"/></g><path d="M688-137c238 0 323-102 359-325h193c-32 297-228 482-552 482-361 0-569-299-569-680 0-197 8-360 73-497 88-186 257-317 513-319 316-2 509 195 535 488h-193c-31-212-118-330-342-330-272 0-397 224-393 523v141c-3 282 115 517 376 517" id="B"/><path d="M497-251l268-831h189L566 0H425L33-1082h189" id="C"/><path d="M538-131c121 0 232-49 232-156 0-50-20-88-56-117-73-60-334-92-420-143-92-55-162-110-162-238 0-190 192-317 400-317 223 0 414 129 413 338H759c0-108-110-186-227-186-119 0-215 53-215 159 0 45 18 78 53 101 76 52 331 90 416 139 98 57 169 115 169 251C955-92 760 20 538 20c-176 0-314-68-386-174-38-55-57-115-57-179h185c6 129 116 202 258 202" id="D"/><g id="e"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#B"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.678385416666668,0)" xlink:href="#t"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,16.282552083333336,0)" xlink:href="#z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.63932291666667,0)" xlink:href="#C"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,30.013020833333336,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,37.07682291666667,0)" xlink:href="#v"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,41.58854166666667,0)" xlink:href="#D"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,48.46354166666667,0)" xlink:href="#y"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,51.69921875000001,0)" xlink:href="#t"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,59.30338541666667,0)" xlink:href="#z"/></g><path d="M730-137c145 0 253-29 323-105v-327H716v-156h528v534C1148-51 964 20 729 20c-394 0-598-279-607-678v-127c-5-410 194-690 585-691 310-1 499 171 537 447h-192c-36-193-151-289-344-289-285 0-393 220-393 528v119c-5 296 140 534 415 534" id="E"/><g id="f"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,9.08203125,0)" xlink:href="#B"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,17.760416666666668,0)" xlink:href="#u"/></g><path d="M1216-1011c-1 297-202 441-512 441H361V0H169v-1456h537c297-6 510 164 510 445zm-192 2c1-178-126-288-307-289H361v571h345c196 1 318-92 318-282" id="F"/><path d="M491 20c-241-1-355-148-355-398v-704h185v699c0 164 67 246 200 246 141 0 235-53 282-158v-787h185V0H812l-4-107C736-22 630 20 491 20" id="G"/><path d="M634-1102c292 0 422 254 422 573 0 298-143 547-420 549-137 0-242-48-317-145L310 0H140v-1536h185v573c75-93 178-139 309-139zm-44 965c207 0 281-175 281-413 0-224-79-395-283-395-122 0-210 57-263 170v468c57 113 145 170 265 170" id="H"/><path d="M177 125H18l608-1581h158" id="I"/><g id="g"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.411458333333334,0)" xlink:href="#G"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.76171875,0)" xlink:href="#H"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.2421875,0)" xlink:href="#I"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,28.736979166666668,0)" xlink:href="#u"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,36.64713541666667,0)" xlink:href="#G"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.997395833333336,0)" xlink:href="#H"/></g><path d="M1222-777c17 480-206 763-650 777H169v-1456h411c408 9 627 269 642 679zm-190 4c3-312-149-522-447-525H361v1141h202c308 0 473-209 469-531v-85" id="J"/><path d="M704-1390c-164-38-297 28-288 194v114h231v143H416V0H231v-939H60v-143h171v-111c-12-274 208-416 483-347" id="K"/><path d="M341 0H156v-1536h185V0" id="L"/><path d="M1098-255l208-827h185L1176 0h-150L763-820 507 0H357L43-1082h184l213 810 252-810h149" id="M"/><g id="h"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,27.604166666666668,0)" xlink:href="#K"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,32.23307291666667,0)" xlink:href="#L"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.46875000000001,0)" xlink:href="#t"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.07291666666667,0)" xlink:href="#M"/></g><path d="M185-189c-5-48-123-54-124 2 14 75 158 14 163 119 3 78-121 87-175 55-17-10-28-26-33-46l33-7c5 56 141 63 141-1 0-78-155-14-162-118-5-82 145-84 179-34 5 7 8 16 11 25" id="N"/><path d="M96-169c-40 0-48 33-48 73s9 75 48 75c24 0 41-14 43-38l32 2c-6 37-31 61-74 61-59 0-76-41-82-99-10-93 101-131 147-64 4 7 5 14 7 22l-32 3c-4-21-16-35-41-35" id="O"/><path d="M141-36C126-15 110 5 73 4 37 3 15-17 15-53c-1-64 63-63 125-63 3-35-9-54-41-54-24 1-41 7-42 31l-33-3c5-37 33-52 76-52 45 0 72 20 72 64v82c-1 20 7 32 28 27v20c-31 9-61-2-59-35zM48-53c0 20 12 33 32 33 41-3 63-29 60-74-43 2-92-5-92 41" id="P"/><path d="M117-194c89-4 53 116 60 194h-32v-121c0-31-8-49-39-48C34-167 62-67 57 0H25l-1-190h30c1 10-1 24 2 32 11-22 29-35 61-36" id="Q"/><g id="i"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#N"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.9259259259259265,0)" xlink:href="#O"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,10.37037037037037,0)" xlink:href="#P"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,15.308641975308642,0)" xlink:href="#Q"/></g><path d="M190 0L58-211 59 0H30v-248h39L202-35l-2-213h31V0h-41" id="R"/><path d="M100-194c63 0 86 42 84 106H49c0 40 14 67 53 68 26 1 43-12 49-29l28 8c-11 28-37 45-77 45C44 4 14-33 15-96c1-61 26-98 85-98zm52 81c6-60-76-77-97-28-3 7-6 17-6 28h103" id="S"/><path d="M206 0h-36l-40-164L89 0H53L-1-190h32L70-26l43-164h34l41 164 42-164h31" id="T"/><g id="j"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#R"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,6.395061728395063,0)" xlink:href="#S"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,11.333333333333336,0)" xlink:href="#T"/></g><path d="M63-220v92h138v28H63V0H30v-248h175v28H63" id="U"/><path d="M24-231v-30h32v30H24zM24 0v-190h32V0H24" id="V"/><path d="M24 0v-261h32V0H24" id="W"/><g id="k"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#U"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.407407407407407,0)" xlink:href="#V"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,7.358024691358025,0)" xlink:href="#W"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,9.308641975308642,0)" xlink:href="#S"/></g><path d="M30-248c87 1 191-15 191 75 0 78-77 80-158 76V0H30v-248zm33 125c57 0 124 11 124-50 0-59-68-47-124-48v98" id="X"/><path d="M84 4C-5 8 30-112 23-190h32v120c0 31 7 50 39 49 72-2 45-101 50-169h31l1 190h-30c-1-10 1-25-2-33-11 22-28 36-60 37" id="Y"/><path d="M115-194c53 0 69 39 70 98 0 66-23 100-70 100C84 3 66-7 56-30L54 0H23l1-261h32v101c10-23 28-34 59-34zm-8 174c40 0 45-34 45-75 0-40-5-75-45-74-42 0-51 32-51 76 0 43 10 73 51 73" id="Z"/><path d="M135-143c-3-34-86-38-87 0 15 53 115 12 119 90S17 21 10-45l28-5c4 36 97 45 98 0-10-56-113-15-118-90-4-57 82-63 122-42 12 7 21 19 24 35" id="aa"/><path d="M106-169C34-169 62-67 57 0H25v-261h32l-1 103c12-21 28-36 61-36 89 0 53 116 60 194h-32v-121c2-32-8-49-39-48" id="ab"/><g id="l"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#X"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.9259259259259265,0)" xlink:href="#Y"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,10.8641975308642,0)" xlink:href="#Z"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,15.80246913580247,0)" xlink:href="#W"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,17.75308641975309,0)" xlink:href="#V"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,19.703703703703706,0)" xlink:href="#aa"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,24.148148148148152,0)" xlink:href="#ab"/></g><path d="M233-177c-1 41-23 64-60 70L243 0h-38l-65-103H63V0H30v-248c88 3 205-21 203 71zM63-129c60-2 137 13 137-47 0-61-80-42-137-45v92" id="ac"/><path d="M100-194c62-1 85 37 85 99 1 63-27 99-86 99S16-35 15-95c0-66 28-99 85-99zM99-20c44 1 53-31 53-75 0-43-8-75-51-75s-53 32-53 75 10 74 51 75" id="ad"/><path d="M114-163C36-179 61-72 57 0H25l-1-190h30c1 12-1 29 2 39 6-27 23-49 58-41v29" id="ae"/><path d="M85-194c31 0 48 13 60 33l-1-100h32l1 261h-30c-2-10 0-23-3-31C134-8 116 4 85 4 32 4 16-35 15-94c0-66 23-100 70-100zm9 24c-40 0-46 34-46 75 0 40 6 74 45 74 42 0 51-32 51-76 0-42-9-74-50-73" id="af"/><g id="m"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#ac"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,6.395061728395063,0)" xlink:href="#S"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,11.333333333333336,0)" xlink:href="#O"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,15.777777777777779,0)" xlink:href="#ad"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,20.71604938271605,0)" xlink:href="#ae"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,23.654320987654323,0)" xlink:href="#af"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,28.592592592592595,0)" xlink:href="#aa"/></g></defs></g></svg>

**In this demo we'll create some artifical fake data to mimic streaming and batch data sources.**


In [0]:
bucket = storage_client.get_bucket(gcs_bucket_name)
streaming_actor_schema = bucket.blob("datagen-schema/streaming-actor.json")
batch_actor_schema = bucket.blob("datagen-schema/batch-actor.json")

**Defining the schema of generated data for both Streaming and Batch data.**

In [0]:
streaming_actor_schema.upload_from_string('''
[
  {"name": "user_id","class":"foreign-key","size": 10000,"skew":0},
  {"name": "actor", "class":"string", "dist":{"streaming-actor":1}},
  {"name": "ts", "class":"current-time"},
  {"name":"v1", "class":"int", "min":1, "max":2},
  {"name": "v2", "class": "random-walk"},
  {"name": "v3", "class": "random-walk","s": 1},
  {"name": "v4", "class": "random-walk", "precision": {"class": "gamma", "dof": 1}},
  {"name": "v5", "class": "gamma", "alpha": 0.1, "beta": 0.1},
  {"name": "v6", "class": "random-walk", "mean":100, "sd":5},
  {"name": "location","class": "zip","near": "37.77,-122.41","milesFrom": 1, "fields":"latitude, longitude, zip"}
]
''')

In [0]:
batch_actor_schema.upload_from_string('''
[
  {"name": "user_id","class":"foreign-key","size": 10000,"skew":0},
  {"name": "actor", "class":"string", "dist":{"batch-actor":1}},
  {"name": "ts", "class":"current-time"},
  {"name": "v1", "class":"sin"},
  {"name": "v2", "class": "random-walk"},
  {"name": "v3", "class": "random-walk","s": 1},
  {"name": "v4", "class": "random-walk", "precision": {"class": "gamma", "dof": 1}},
  {"name": "v5", "class": "gamma", "alpha": 0.1, "beta": 0.1},
  {"name": "v6", "class": "random-walk", "mean":100, "sd":5},
  {"name": "location","class": "zip","near": "37.77,-122.41","milesFrom": 1, "fields":"latitude, longitude, zip"}
]
''')

**Uploading datagenerator schemas to GCS**

In [0]:
for blob in bucket.list_blobs(prefix="datagen-schema"):
  print(blob)

**Downloading data generator code**

In [0]:
!wget https://www-us.apache.org/dist/maven/maven-3/3.6.0/binaries/apache-maven-3.6.0-bin.tar.gz; tar -xzf apache-maven-3.6.0-bin.tar.gz

In [0]:
!git clone https://github.com/pdeyhim/log-synth.git /content/log-synth

In [0]:
cd /content/log-synth

In [0]:
!/content/apache-maven-3.6.0/bin/mvn  install -Dmaven.test.skip=true

In [0]:
!git clone https://github.com/pdeyhim/datagenerator.git /content/datagenerator/

In [0]:
cd /content/datagenerator/

**Starting streaming data generator Dataflow job**

In [0]:
streaming_datagen_mvn_command = '''/content/apache-maven-3.6.0/bin/mvn compile exec:java \
 -Dexec.mainClass=com.google.datagenerator.SampleDataGeneratorStreaming \
 -Dexec.cleanupDaemonThreads=false \
 -Dexec.args=" \
 --project={project_name} \
 --stagingLocation=gs://{gcs_bucket_name}/dataflow/pipelines/sampler/staging \
 --tempLocation=gs://{gcs_bucket_name}/dataflow/pipelines/sampler/temp \
 --runner=DataflowRunner \
 --windowDuration=5s \
 --numShards=5 \
 --qps=100 \
 --schemaLocation=gs://{gcs_bucket_name}/datagen-schema/streaming-actor.json \
 --outputTopic=projects/{project_name}/topics/{data_source_topic_name} \
 --autoscalingAlgorithm=THROUGHPUT_BASED \
 --maxNumWorkers=3"
'''.format(gcs_bucket_name=gcs_bucket_name,project_name=project_name,data_source_topic_name=data_source_topic_name)

out = subprocess.run(streaming_datagen_mvn_command,shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out.stdout.decode().split("\n")

**Starting batch data generator Dataflow job**

In [0]:
batch_datagen_mvn_command = '''/content/apache-maven-3.6.0/bin/mvn compile exec:java \
 -Dexec.mainClass=com.google.datagenerator.SampleDataGeneratorBatch \
 -Dexec.cleanupDaemonThreads=false \
 -Dexec.args=" \
 --project={project_name} \
 --stagingLocation=gs://{gcs_bucket_name}/dataflow/pipelines/sampler/staging \
 --tempLocation=gs://{gcs_bucket_name}/dataflow/pipelines/sampler/temp \
 --runner=DataflowRunner \
 --windowDuration=10m \
 --numShards=5 \
 --qps=1000 \
 --schemaLocation=gs://{gcs_bucket_name}/datagen-schema/batch-actor.json \
 --outputDirectory=gs://{gcs_bucket_name}/dataflow-sampler/data/ \
 --outputFilenamePrefix=user-simulated-data- \
 --outputFilenameSuffix=.json \
 --autoscalingAlgorithm=THROUGHPUT_BASED \
 --maxNumWorkers=3"'''.format(gcs_bucket_name=gcs_bucket_name,project_name=project_name )

out = subprocess.run(batch_datagen_mvn_command,shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out.stdout.decode().split("\n")

**Once the commands above have successfully executed you should be able to view the progress of each Dataflow job in the consose: **https://pantheon.corp.google.com/dataflow

# Converting Batch to Streaming Data



In [0]:
%%html
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:lucid="lucid" width="400.43" height="443.72"><g transform="translate(-480 -140)" lucid:page-tab-id="pCGVoe.yl8lM"><path d="M500 162c0-1.1.9-2 2-2h196c1.1 0 2 .9 2 2v316c0 1.1-.9 2-2 2H502c-1.1 0-2-.9-2-2z" fill="#ffebee"/><use xlink:href="#a" transform="matrix(1,0,0,1,516,176) translate(0 12.369791666666668)"/><use xlink:href="#b" transform="matrix(1,0,0,1,516,176) translate(15.227864583333336 12.369791666666668)"/><use xlink:href="#c" transform="matrix(1,0,0,1,516,176) translate(71.62109375 12.369791666666668)"/><use xlink:href="#d" transform="matrix(1,0,0,1,516,176) translate(119.55078125 12.369791666666668)"/><use xlink:href="#e" transform="matrix(1,0,0,1,516,176) translate(0 27.99479166666667)"/><path d="M540 229.73c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v96c0 1.1-.9 2-2 2H542c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M552.5 227.73h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.58.6 1.43v2m0 8v4m0 8v4m0 8v4m0 8v4m0 8v4m0 8v4m0 8v4m0 8v2l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0H542l-1.4-.6-.6-1.4v-2m0-8v-4m0-8v-4m0-8v-4m0-8v-4m0-8v-4m0-8v-4m0-8v-4m0-8v-2l.6-1.42 1.4-.57h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M586.73 283.77l-9.63-18.4c-.4-.72-.58-1.5-.58-2.33 0-.83.2-1.6.58-2.32l9.63-18.4c.38-.73.9-1.3 1.55-1.7.65-.42 1.36-.62 2.1-.62h19.23c.76 0 1.46.22 2.12.63.65.4 1.16.97 1.55 1.68l9.63 18.42c.4.7.58 1.5.58 2.32 0 .84-.2 1.6-.58 2.33l-9.63 18.4c-.38.72-.9 1.3-1.55 1.7-.65.4-1.36.6-2.1.6h-19.27c-.75.03-1.45-.17-2.1-.58-.66-.43-1.16-1-1.52-1.73zm0 0" fill="#0e62c2"/><path d="M605.95 251.58l-5.93 1.57-6.04-1.54 3 3.33-5.05 5.8.9 1-.86 2.4 1.82 2-1.87 2.1 4.65 5.15-2.6 1.25 10.4 11.5h5.2c.75-.02 1.45-.22 2.1-.63.65-.4 1.17-.98 1.56-1.7l8-15.32z" fill="#0851b0"/><path d="M605.7 263.1c0-.8.58-1.45 1.3-1.45.74 0 1.33.65 1.33 1.45 0 .8-.6 1.46-1.32 1.46-.72 0-1.3-.66-1.3-1.46zM591.63 263.1c0-.8.6-1.45 1.32-1.45.73 0 1.32.65 1.32 1.45 0 .8-.6 1.46-1.32 1.46-.73 0-1.32-.66-1.32-1.46zM598.66 260.96c0-.8.6-1.46 1.32-1.46.73 0 1.32.65 1.32 1.46 0 .8-.6 1.45-1.32 1.45-.73 0-1.32-.64-1.32-1.44zM598.66 265.3c0-.82.6-1.47 1.32-1.47.73 0 1.32.65 1.32 1.46 0 .8-.6 1.44-1.32 1.44-.73 0-1.32-.65-1.32-1.45z" fill="#fff"/><path d="M601.96 254.16l4.03-2.58-1.95-1.22h-.08l-4 2.55-3.98-2.54h-.08l-1.94 1.22 4.02 2.58-6.07 3.84v2.72H594v-1.46l5-3.2v2.35h1.97v-2.34l4.98 3.2v1.46H608V258zM605.95 266.95l-4.98 3.23v-2.34H599v2.34l-5-3.23v-1.46h-2.07v2.7l6.07 3.88-4.02 2.55 1.94 1.25h.07l3.98-2.55 4 2.55h.07l1.94-1.25-4.04-2.55 6.04-3.88v-2.7h-2.05z" fill="#fff"/><use xlink:href="#f" transform="matrix(1,0,0,1,576.5217391304348,291.1187500000001) translate(6.384114583333332 14.557291666666668)"/><path d="M743.04 233.82c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v87.82c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M755.54 231.82h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.58.6 1.42v2.1m0 8.35v4.18m0 8.37V261m0 8.36v4.2m0 8.35v4.2m0 8.35v4.2m0 8.35v4.18m0 8.37v2.1l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.6-.6-1.4v-2.1m0-8.37V307m0-8.36v-4.2m0-8.35v-4.2m0-8.35v-4.2m0-8.35v-4.18m0-8.37v-4.18m0-8.36v-2.08l.6-1.42 1.4-.58h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M788.48 285l-9.64-16.85c-.4-.65-.58-1.36-.58-2.13 0-.77.2-1.48.58-2.13l9.6-16.9c.38-.67.9-1.2 1.55-1.58.65-.4 1.35-.58 2.1-.57h19.27c.75 0 1.46.2 2.1.58.67.4 1.18.9 1.56 1.57l9.6 16.9c.4.64.6 1.35.6 2.12s-.2 1.48-.6 2.13l-9.6 16.9c-.38.65-.9 1.17-1.55 1.56-.65.4-1.36.58-2.1.6h-19.23c-.76-.03-1.46-.24-2.1-.62-.67-.4-1.18-.92-1.57-1.58zm0 0" fill="#0e62c2"/><path d="M809.64 261l-13.27 1.22-2.57 2.6 2.2 2.22-2.16 3.9 16 16.17h1.53c.75 0 1.46-.18 2.1-.57.67-.38 1.18-.9 1.56-1.57l6.7-11.72z" fill="#0851b0"/><path d="M809.35 260.92h-15.18c-.08 0-.16.02-.23.05-.07.04-.12.1-.16.17-.04.07-.06.14-.05.22v3.16c0 .08 0 .15.05.22.04.07.1.13.16.17.07.05.15.07.23.06h15.18c.08 0 .15 0 .22-.05.07-.03.13-.1.17-.16.03-.07.05-.14.05-.22v-3.16c0-.08-.03-.15-.06-.22-.04-.07-.1-.13-.17-.17-.07-.03-.14-.06-.22-.05zM807 263.9c-.25 0-.47-.1-.65-.28-.17-.2-.27-.4-.27-.66 0-.25.1-.48.27-.66.18-.18.4-.27.65-.27.26 0 .47.1.65.27.18.18.27.4.27.66 0 .25-.1.47-.27.65-.18.2-.4.28-.65.3zm0 0M809.35 267h-15.18c-.13 0-.23.05-.3.13-.1.1-.14.2-.14.32v3.2c0 .07 0 .14.05.22.04.07.1.12.16.16.07.04.15.06.23.06h15.18c.08 0 .15-.03.22-.07.07-.04.13-.1.17-.16.03-.08.05-.15.05-.23v-3.15c0-.14-.05-.25-.14-.34-.08-.1-.2-.15-.3-.16zm-2.35 3c-.25 0-.47-.08-.65-.26-.17-.18-.27-.4-.27-.66 0-.25.1-.47.27-.65.18-.2.4-.28.65-.28.26 0 .47.1.65.28.18.18.27.4.27.65 0 .26-.1.47-.27.65-.18.2-.4.28-.65.28zm0 0" fill="#fff"/><use xlink:href="#g" transform="matrix(1,0,0,1,778.2608695652173,292.1891613142292) translate(10.664713541666666 14.557291666666668)"/><path d="M740.17 473.5c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v88.22c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M752.67 471.5h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v2.1m0 8.4v4.2m0 8.4v4.2m0 8.4v4.2m0 8.4v4.22m0 8.4v4.2m0 8.4v4.2m0 8.4v2.1l-.6 1.42-1.4.58h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.58-.6-1.42v-2.1m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4V484m0-8.4v-2.1l.6-1.43 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M785.6 532.7l-9.63-20.14c-.4-.78-.58-1.63-.58-2.55 0-.9.18-1.76.57-2.54l9.6-20.2c.38-.8.9-1.43 1.55-1.9.66-.45 1.36-.68 2.12-.67h19.26c.75 0 1.46.22 2.1.68.67.46 1.18 1.1 1.56 1.88l9.6 20.2c.4.78.6 1.63.6 2.55 0 .93-.2 1.78-.6 2.56l-9.6 20.2c-.38.8-.9 1.4-1.55 1.88-.64.46-1.35.68-2.1.7h-19.22c-.75-.04-1.46-.28-2.1-.75-.66-.47-1.18-1.1-1.57-1.9zm0 0" fill="#0e62c2"/><path d="M808.13 502.75l-2.13.58-5.35-6.48-1.72 3.86-.92 4.5 1.58 1.9-3 .76-4.18-5.02-2.83 3.28 5.17 6.26-5.5 5.28 14.56 17.62h4.7c.75 0 1.46-.23 2.1-.7.67-.45 1.18-1.07 1.56-1.87l7.56-15.88z" fill="#0851b0"/><path d="M805.1 505.66c-.54-1.12-.22-2.56.7-3.2.93-.66 2.12-.27 2.66.85.53 1.14.22 2.58-.7 3.22-.94.65-2.13.27-2.67-.86zM790.03 506.5c-.93-.64-1.24-2.08-.7-3.2.53-1.13 1.72-1.5 2.65-.86.93.65 1.24 2.08.7 3.2-.53 1.13-1.72 1.52-2.65.87zM796.94 521c0-1.3.87-2.36 1.95-2.36 1.06 0 1.93 1.06 1.93 2.36 0 1.3-.87 2.35-1.94 2.35-1.1 0-1.96-1.06-1.96-2.35zM798.44 510.94l.9-1.9 7.87 5.5-.88 1.9z" fill="#fff"/><path d="M790.55 514.58l7.87-5.5.9 1.88-7.87 5.5z" fill="#fff"/><path d="M797.97 498.98h1.8v11h-1.8z" fill="#fff"/><path d="M795.73 509.98c0-2.1 1.42-3.8 3.16-3.8 1.73 0 3.14 1.7 3.14 3.8 0 2.1-1.4 3.82-3.15 3.82s-3.17-1.7-3.17-3.82zM788.84 517c-.7-1.45-.28-3.3.92-4.13 1.2-.83 2.7-.34 3.4 1.1.7 1.45.28 3.3-.9 4.13-1.2.83-2.73.34-3.42-1.1zM805.52 518.1c-1.2-.83-1.6-2.67-.9-4.1.68-1.46 2.2-1.95 3.4-1.12 1.18.84 1.6 2.68.9 4.12-.68 1.45-2.2 1.94-3.4 1.1zM796.4 498.98c0-1.67 1.1-3.02 2.5-3.02 1.36 0 2.48 1.35 2.48 3.02 0 1.66-1.12 3.02-2.5 3.02-1.37 0-2.5-1.36-2.5-3.02z" fill="#fff"/><use xlink:href="#h" transform="matrix(1,0,0,1,775.391304347826,540.3289711850528) translate(-2.238932291666668 14.557291666666668)"/><path d="M658.4 280.66h41.82v-.24h26.5" stroke="#000" stroke-width=".5" fill="none"/><path d="M658.4 280.9h-1v-.5h1z"/><path d="M741.23 280.42l-14.26 4.64v-9.27z" stroke="#000" stroke-width=".5"/><use xlink:href="#i" transform="matrix(1,0,0,1,678.2714576338336,265.75828457047186) translate(0 7.111111111111112)"/><use xlink:href="#j" transform="matrix(1,0,0,1,678.2714576338336,265.75828457047186) translate(25.185185185185187 7.111111111111112)"/><path d="M540 348.42c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v101c0 1.1-.9 2-2 2H542c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M552.5 346.42h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v2.1m0 8.42v4.2m0 8.43v4.2m0 8.42v4.2m0 8.42v4.2m0 8.42v4.2m0 8.43v4.2m0 8.42v4.2m0 8.42v2.1l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0H542l-1.4-.6-.6-1.4v-2.1m0-8.42v-4.2m0-8.43v-4.2m0-8.42v-4.2m0-8.43v-4.2m0-8.42v-4.2m0-8.43v-4.2m0-8.42v-4.2m0-8.43v-2.1l.6-1.4 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M585.43 405.5l-9.64-19.32c-.4-.75-.58-1.57-.58-2.44 0-.88.2-1.7.57-2.44l9.63-19.33c.38-.75.9-1.35 1.55-1.78.65-.44 1.35-.65 2.1-.65h19.23c.76 0 1.46.22 2.1.65.66.44 1.18 1.03 1.57 1.77l9.63 19.33c.38.75.57 1.56.57 2.44 0 .87-.2 1.7-.57 2.44l-9.63 19.33c-.38.76-.9 1.36-1.55 1.8-.66.42-1.36.63-2.1.63h-19.27c-.76.02-1.45-.2-2.1-.62-.66-.43-1.16-1.03-1.52-1.8zm0 0" fill="#0e62c2"/><path d="M604.65 371.7l-5.94 1.66-6.03-1.62 3 3.5-5.05 6.06.92 1.06-.88 2.5 1.83 2.13-1.88 2.2 4.65 5.4-2.6 1.3 10.4 12.07h5.2c.75 0 1.45-.22 2.1-.65.66-.43 1.18-1.03 1.56-1.77l8.02-16.1z" fill="#0851b0"/><path d="M604.4 383.8c0-.84.58-1.52 1.3-1.52.73 0 1.33.68 1.33 1.52s-.6 1.53-1.32 1.53c-.72 0-1.3-.7-1.3-1.53zM590.33 383.8c0-.84.6-1.52 1.32-1.52.73 0 1.32.68 1.32 1.52s-.6 1.53-1.32 1.53c-.73 0-1.32-.7-1.32-1.53zM597.36 381.55c0-.84.6-1.53 1.32-1.53.72 0 1.32.7 1.32 1.53 0 .84-.6 1.53-1.32 1.53-.73 0-1.32-.7-1.32-1.53zM597.36 386.1c0-.85.6-1.53 1.32-1.53.72 0 1.32.68 1.32 1.53 0 .84-.6 1.53-1.32 1.53-.73 0-1.32-.7-1.32-1.53z" fill="#fff"/><path d="M600.65 374.42l4.03-2.72-1.94-1.28h-.07l-4 2.68-3.98-2.68h-.08l-1.95 1.28 4.03 2.72-6.08 4.03v2.85h2.1v-1.53l4.97-3.36v2.48h1.97v-2.47l4.98 3.37v1.53h2.05v-2.85zM604.65 387.84l-4.98 3.4v-2.47h-1.98v2.46l-5-3.4v-1.52h-2.08v2.85l6.08 4.08-4.03 2.68 1.95 1.32h.07l3.98-2.67 4 2.67h.06l1.94-1.3-4.03-2.7 6.05-4.07v-2.84h-2.05z" fill="#fff"/><use xlink:href="#k" transform="matrix(1,0,0,1,575.2173913043478,412.97014204545457) translate(-6.864583333333339 14.557291666666668)"/><path d="M598.7 452.42v65.27h125.15" stroke="#000" stroke-width=".5" fill="none"/><path d="M598.95 452.42h-.5v-1h.5z"/><path d="M738.36 517.7l-14.26 4.63v-9.27z" stroke="#000" stroke-width=".5"/><path d="M622.17 499.67h74.6v20.67h-74.6z" fill="#fff" fill-opacity="0"/><g><use xlink:href="#l" transform="matrix(1,0,0,1,627.1739130434783,504.67084381763465) translate(0 7.111111111111112)"/><use xlink:href="#m" transform="matrix(1,0,0,1,627.1739130434783,504.67084381763465) translate(31.55555555555556 7.111111111111112)"/></g><defs><path d="M973-380H363L226 0H28l556-1456h168L1309 0h-197zM421-538h495l-248-681" id="n"/><path d="M248 11c-71 0-114-43-114-108 0-67 42-112 114-112s116 45 116 112c-1 65-45 108-116 108zm1-883c-71 0-114-43-114-108 0-67 42-112 114-112s116 45 116 112c-1 65-45 108-116 108" id="o"/><g id="a"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#n"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.697916666666668,0)" xlink:href="#o"/></g><path d="M1222-777c17 480-206 763-650 777H169v-1456h411c408 9 627 269 642 679zm-190 4c3-312-149-522-447-525H361v1141h202c308 0 473-209 469-531v-85" id="p"/><path d="M561-1102c238-4 403 126 403 351v498c0 99 13 178 38 237V0H808c-11-21-19-59-26-114C696-25 593 20 474 20c-199 0-368-130-365-320 4-251 214-359 490-356h180v-85c-1-135-86-212-229-212-115 0-232 67-233 171H131c20-205 206-316 430-320zM294-326c0 117 90 185 207 185 122 0 239-75 278-162v-222H634c-227 0-340 66-340 199" id="q"/><path d="M456 20C285 20 205-92 206-268v-671H9v-143h197v-262h185v262h202v143H391v671c-9 125 92 149 207 118V0c-49 13-96 20-142 20" id="r"/><path d="M704-1390c-164-38-297 28-288 194v114h231v143H416V0H231v-939H60v-143h171v-111c-12-274 208-416 483-347" id="s"/><path d="M341 0H156v-1536h185V0" id="t"/><path d="M584 20C278 26 81-227 91-551c10-320 175-545 491-551 308-5 503 247 494 573-9 322-175 543-492 549zm-2-970c-208 0-305 185-305 421 0 216 106 398 307 398 211 0 307-186 307-420 0-214-109-399-309-399" id="u"/><path d="M1098-255l208-827h185L1176 0h-150L763-820 507 0H357L43-1082h184l213 810 252-810h149" id="v"/><g id="b"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#r"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,27.604166666666668,0)" xlink:href="#s"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,32.23307291666667,0)" xlink:href="#t"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.46875000000001,0)" xlink:href="#u"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.07291666666667,0)" xlink:href="#v"/></g><path d="M631-137c179 0 314-70 314-231 0-71-25-127-75-166s-141-77-272-115c-165-47-285-106-360-175s-112-154-112-255c0-245 231-397 495-397 206 0 369 89 449 218 43 68 65 142 65 223H942c0-184-129-283-321-283-172 0-299 76-301 236 0 59 26 109 76 150s135 78 256 113c205 59 345 116 436 256 35 53 50 119 50 193-2 256-231 398-507 390C333 12 90-129 80-422h193c0 187 160 285 358 285" id="w"/><path d="M1240-945c-141 0-250 95-250 227V0H804v-709c0-157-77-236-231-236-121 0-204 52-249 155V0H139v-1082h175l5 120c79-93 186-140 321-140 151 0 254 58 309 174 69-98 181-174 345-174 237 0 362 126 362 377V0h-185v-714c-2-159-67-231-231-231" id="x"/><path d="M632-1102c291 0 422 251 422 573 0 297-141 548-419 549-131 0-235-42-310-125v521H140v-1498h169l9 120c75-93 180-140 314-140zm-53 971c207 0 290-180 290-419 0-218-92-395-292-395-112 0-196 50-252 149v517c55 99 140 148 254 148" id="y"/><path d="M599-131c141 0 220-65 285-146l113 88C906-50 770 20 589 20 281 21 93-214 93-545c0-223 93-397 233-485 74-48 154-72 240-72 300 2 449 218 445 537v77H278c0 197 129 357 321 357zm227-509c-3-180-88-310-260-310-170 0-264 140-282 310h542" id="z"/><g id="c"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,7.91015625,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.162760416666668,0)" xlink:href="#x"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,26.848958333333336,0)" xlink:href="#y"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,34.329427083333336,0)" xlink:href="#t"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,37.56510416666667,0)" xlink:href="#z"/></g><g id="d"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#r"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#q"/></g><path d="M730-137c145 0 253-29 323-105v-327H716v-156h528v534C1148-51 964 20 729 20c-394 0-598-279-607-678v-127c-5-410 194-690 585-691 310-1 499 171 537 447h-192c-36-193-151-289-344-289-285 0-393 220-393 528v119c-5 296 140 534 415 534" id="A"/><path d="M589-945c-131 0-219 81-264 174V0H140v-1082h175l6 136c83-104 191-156 324-156 229 0 346 129 346 387V0H806v-716c-2-153-65-229-217-229" id="B"/><path d="M663-916c-163-27-288 18-338 148V0H140v-1082h180l3 125c61-97 147-145 258-145 36 0 63 5 82 14v172" id="C"/><path d="M538-131c121 0 232-49 232-156 0-50-20-88-56-117-73-60-334-92-420-143-92-55-162-110-162-238 0-190 192-317 400-317 223 0 414 129 413 338H759c0-108-110-186-227-186-119 0-215 53-215 159 0 45 18 78 53 101 76 52 331 90 416 139 98 57 169 115 169 251C955-92 760 20 538 20c-176 0-314-68-386-174-38-55-57-115-57-179h185c6 129 116 202 258 202" id="D"/><g id="e"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#A"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,9.08203125,0)" xlink:href="#z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,16.145833333333336,0)" xlink:href="#B"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.50260416666667,0)" xlink:href="#z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,30.566406250000007,0)" xlink:href="#C"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,34.81770833333334,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.07031250000001,0)" xlink:href="#r"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,46.29557291666667,0)" xlink:href="#u"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,53.899739583333336,0)" xlink:href="#C"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,58.411458333333336,0)" xlink:href="#D"/></g><path d="M1160-420C1159-145 963 0 674 0H169v-1456h476c294-1 476 112 476 388 0 153-99 253-216 302 146 38 255 167 255 346zM678-157c171 2 289-95 289-261 0-175-95-263-286-263H361v524h317zm250-912c-2-172-104-229-283-229H361v463h290c157 3 278-84 277-234" id="E"/><path d="M277-555c0 244 77 420 297 424 127 2 249-93 255-210h175C980-127 805 20 574 20 258 20 81-222 92-562c11-319 164-533 481-540 237-5 426 165 431 392H829c-7-133-115-242-256-240-209 4-296 166-296 395" id="F"/><path d="M589-945c-131 0-219 81-264 174V0H140v-1536h185v585c82-101 189-151 320-151 229 0 346 129 346 387V0H806v-716c-2-153-65-229-217-229" id="G"/><g id="f"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.553385416666668,0)" xlink:href="#r"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,19.908854166666668,0)" xlink:href="#F"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,26.888020833333336,0)" xlink:href="#G"/></g><path d="M688-137c238 0 323-102 359-325h193c-32 297-228 482-552 482-361 0-569-299-569-680 0-197 8-360 73-497 88-186 257-317 513-319 316-2 509 195 535 488h-193c-31-212-118-330-342-330-272 0-397 224-393 523v141c-3 282 115 517 376 517" id="H"/><g id="g"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#A"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,9.08203125,0)" xlink:href="#H"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,17.760416666666668,0)" xlink:href="#w"/></g><path d="M1216-1011c-1 297-202 441-512 441H361V0H169v-1456h537c297-6 510 164 510 445zm-192 2c1-178-126-288-307-289H361v571h345c196 1 318-92 318-282" id="I"/><path d="M491 20c-241-1-355-148-355-398v-704h185v699c0 164 67 246 200 246 141 0 235-53 282-158v-787h185V0H812l-4-107C736-22 630 20 491 20" id="J"/><path d="M634-1102c292 0 422 254 422 573 0 298-143 547-420 549-137 0-242-48-317-145L310 0H140v-1536h185v573c75-93 178-139 309-139zm-44 965c207 0 281-175 281-413 0-224-79-395-283-395-122 0-210 57-263 170v468c57 113 145 170 265 170" id="K"/><path d="M177 125H18l608-1581h158" id="L"/><g id="h"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#I"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.411458333333334,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.76171875,0)" xlink:href="#K"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.2421875,0)" xlink:href="#L"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,28.736979166666668,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,36.64713541666667,0)" xlink:href="#J"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.997395833333336,0)" xlink:href="#K"/></g><path d="M160-131c35 5 61 23 61 61C221 17 115-2 30 0v-248c76 3 177-17 177 60 0 33-19 50-47 57zm-97-11c50-1 110 9 110-42 0-47-63-36-110-37v79zm0 115c55-2 124 14 124-45 0-56-70-42-124-44v89" id="M"/><path d="M141-36C126-15 110 5 73 4 37 3 15-17 15-53c-1-64 63-63 125-63 3-35-9-54-41-54-24 1-41 7-42 31l-33-3c5-37 33-52 76-52 45 0 72 20 72 64v82c-1 20 7 32 28 27v20c-31 9-61-2-59-35zM48-53c0 20 12 33 32 33 41-3 63-29 60-74-43 2-92-5-92 41" id="N"/><path d="M59-47c-2 24 18 29 38 22v24C64 9 27 4 27-40v-127H5v-23h24l9-43h21v43h35v23H59v120" id="O"/><path d="M96-169c-40 0-48 33-48 73s9 75 48 75c24 0 41-14 43-38l32 2c-6 37-31 61-74 61-59 0-76-41-82-99-10-93 101-131 147-64 4 7 5 14 7 22l-32 3c-4-21-16-35-41-35" id="P"/><path d="M106-169C34-169 62-67 57 0H25v-261h32l-1 103c12-21 28-36 61-36 89 0 53 116 60 194h-32v-121c2-32-8-49-39-48" id="Q"/><g id="i"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#M"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.9259259259259265,0)" xlink:href="#N"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,10.8641975308642,0)" xlink:href="#O"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,13.333333333333334,0)" xlink:href="#P"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,17.77777777777778,0)" xlink:href="#Q"/></g><path d="M63-220v92h138v28H63V0H30v-248h175v28H63" id="R"/><path d="M24-231v-30h32v30H24zM24 0v-190h32V0H24" id="S"/><path d="M24 0v-261h32V0H24" id="T"/><path d="M100-194c63 0 86 42 84 106H49c0 40 14 67 53 68 26 1 43-12 49-29l28 8c-11 28-37 45-77 45C44 4 14-33 15-96c1-61 26-98 85-98zm52 81c6-60-76-77-97-28-3 7-6 17-6 28h103" id="U"/><path d="M135-143c-3-34-86-38-87 0 15 53 115 12 119 90S17 21 10-45l28-5c4 36 97 45 98 0-10-56-113-15-118-90-4-57 82-63 122-42 12 7 21 19 24 35" id="V"/><g id="j"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#R"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.407407407407407,0)" xlink:href="#S"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,7.358024691358025,0)" xlink:href="#T"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,9.308641975308642,0)" xlink:href="#U"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,14.246913580246915,0)" xlink:href="#V"/></g><path d="M341 0H156v-1082h185V0zm-91-1264c-68 0-108-42-109-105 0-62 41-107 109-107s110 44 110 107-42 105-110 105" id="W"/><path d="M521 20C240 20 96-239 96-550c0-303 147-552 427-552 132 0 235 47 309 140l9-120h169V-26c4 277-183 454-459 452-165-1-334-81-400-187l96-111c79 98 176 147 291 147 180 1 288-109 286-290v-93C750-23 649 20 521 20zm48-965c-209 0-287 178-287 416 0 217 86 392 285 392 117 0 202-53 257-159v-494c-57-103-142-155-255-155" id="X"/><g id="k"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,7.91015625,0)" xlink:href="#r"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,12.265625,0)" xlink:href="#C"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,16.653645833333336,0)" xlink:href="#z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.71744791666667,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,30.97005208333334,0)" xlink:href="#x"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.65625000000001,0)" xlink:href="#W"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,45.89192708333334,0)" xlink:href="#B"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,53.24869791666668,0)" xlink:href="#X"/></g><path d="M30-248c87 1 191-15 191 75 0 78-77 80-158 76V0H30v-248zm33 125c57 0 124 11 124-50 0-59-68-47-124-48v98" id="Y"/><path d="M84 4C-5 8 30-112 23-190h32v120c0 31 7 50 39 49 72-2 45-101 50-169h31l1 190h-30c-1-10 1-25-2-33-11 22-28 36-60 37" id="Z"/><path d="M115-194c53 0 69 39 70 98 0 66-23 100-70 100C84 3 66-7 56-30L54 0H23l1-261h32v101c10-23 28-34 59-34zm-8 174c40 0 45-34 45-75 0-40-5-75-45-74-42 0-51 32-51 76 0 43 10 73 51 73" id="aa"/><g id="l"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#Y"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,5.9259259259259265,0)" xlink:href="#Z"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,10.8641975308642,0)" xlink:href="#aa"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,15.80246913580247,0)" xlink:href="#T"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,17.75308641975309,0)" xlink:href="#S"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,19.703703703703706,0)" xlink:href="#V"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,24.148148148148152,0)" xlink:href="#Q"/></g><path d="M233-177c-1 41-23 64-60 70L243 0h-38l-65-103H63V0H30v-248c88 3 205-21 203 71zM63-129c60-2 137 13 137-47 0-61-80-42-137-45v92" id="ab"/><path d="M100-194c62-1 85 37 85 99 1 63-27 99-86 99S16-35 15-95c0-66 28-99 85-99zM99-20c44 1 53-31 53-75 0-43-8-75-51-75s-53 32-53 75 10 74 51 75" id="ac"/><path d="M114-163C36-179 61-72 57 0H25l-1-190h30c1 12-1 29 2 39 6-27 23-49 58-41v29" id="ad"/><path d="M85-194c31 0 48 13 60 33l-1-100h32l1 261h-30c-2-10 0-23-3-31C134-8 116 4 85 4 32 4 16-35 15-94c0-66 23-100 70-100zm9 24c-40 0-46 34-46 75 0 40 6 74 45 74 42 0 51-32 51-76 0-42-9-74-50-73" id="ae"/><g id="m"><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,0,0)" xlink:href="#ab"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,6.395061728395063,0)" xlink:href="#U"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,11.333333333333336,0)" xlink:href="#P"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,15.777777777777779,0)" xlink:href="#ac"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,20.71604938271605,0)" xlink:href="#ad"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,23.654320987654323,0)" xlink:href="#ae"/><use transform="matrix(0.02469135802469136,0,0,0.02469135802469136,28.592592592592595,0)" xlink:href="#V"/></g></defs></g></svg>

In [0]:
!git clone https://github.com/GoogleCloudPlatform/DataflowTemplates.git /content/DataflowTemplates

In [0]:
cd /content/DataflowTemplates

In [0]:
 gcs_to_pubsub = '''/content/apache-maven-3.6.0/bin/mvn compile exec:java \
-Dexec.mainClass=com.google.cloud.teleport.templates.TextToPubsubStream \
-Dexec.args=" \
--project={project_name} \
--stagingLocation=gs://{gcs_bucket_name}/dataflow/pipelines/gcstopubsub/staging \
--tempLocation=gs://{gcs_bucket_name}/dataflow/pipelines/gcstopubsub/temp2 \
--runner=DataflowRunner \
--inputFilePattern=gs://{gcs_bucket_name}/dataflow-sampler/data/user-simulated-data-* \
--outputTopic=projects/{project_name}/topics/{data_source_topic_name}"'''.format(gcs_bucket_name=gcs_bucket_name,project_name=project_name,data_source_topic_name=data_source_topic_name)

out = subprocess.run(gcs_to_pubsub,shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out.stdout.decode().split("\n")

# Streaming Data to BigQuery

In [0]:
%%html 
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:lucid="lucid" width="237.74" height="471.42"><g transform="translate(-482.86956521739114 -280)" lucid:page-tab-id="zCGVpFzEY-FB"><path d="M502.87 302c0-1.1.9-2 2-2H698.6c1.1 0 2 .9 2 2v427.42c0 1.1-.9 2-2 2H504.88c-1.1 0-2-.9-2-2z" fill="#e0f2f1"/><use xlink:href="#a" transform="matrix(1,0,0,1,518.8695652173911,316) translate(0 12.369791666666668)"/><use xlink:href="#b" transform="matrix(1,0,0,1,518.8695652173911,316) translate(15.208333333333336 12.369791666666668)"/><use xlink:href="#c" transform="matrix(1,0,0,1,518.8695652173911,316) translate(46.11328125 12.369791666666668)"/><use xlink:href="#d" transform="matrix(1,0,0,1,518.8695652173911,316) translate(86.171875 12.369791666666668)"/><use xlink:href="#e" transform="matrix(1,0,0,1,518.8695652173911,316) translate(101.30208333333333 12.369791666666668)"/><path d="M540.17 353.5c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v88.22c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M552.67 351.5h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v2.1m0 8.4v4.2m0 8.4v4.2m0 8.4v4.2m0 8.4v4.22m0 8.4v4.2m0 8.4v4.2m0 8.4v2.1l-.6 1.42-1.4.58h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.58-.6-1.42v-2.1m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4v-4.2m0-8.4V364m0-8.4v-2.1l.6-1.43 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M585.6 412.7l-9.63-20.14c-.4-.78-.58-1.63-.58-2.55 0-.9.18-1.76.57-2.54l9.6-20.2c.38-.8.9-1.43 1.55-1.9.66-.45 1.36-.68 2.12-.67h19.26c.75 0 1.46.22 2.1.68.67.46 1.18 1.1 1.56 1.88l9.6 20.2c.4.78.6 1.63.6 2.55 0 .93-.2 1.78-.6 2.56l-9.6 20.2c-.38.8-.9 1.4-1.55 1.88-.64.46-1.35.68-2.1.7h-19.22c-.75-.04-1.46-.28-2.1-.75-.66-.47-1.18-1.1-1.57-1.9zm0 0" fill="#0e62c2"/><path d="M608.13 382.75l-2.13.58-5.35-6.48-1.72 3.86-.92 4.5 1.58 1.9-3 .76-4.18-5.02-2.83 3.28 5.17 6.26-5.5 5.28 14.56 17.62h4.7c.75 0 1.46-.23 2.1-.7.67-.45 1.18-1.07 1.56-1.87l7.56-15.88z" fill="#0851b0"/><path d="M605.1 385.66c-.54-1.12-.22-2.56.7-3.2.93-.66 2.12-.27 2.66.85.53 1.14.22 2.58-.7 3.22-.94.65-2.13.27-2.67-.86zM590.03 386.5c-.93-.64-1.24-2.08-.7-3.2.53-1.13 1.72-1.5 2.65-.86.93.65 1.24 2.08.7 3.2-.53 1.13-1.72 1.52-2.65.87zM596.94 401c0-1.3.87-2.36 1.95-2.36 1.06 0 1.93 1.06 1.93 2.36 0 1.3-.87 2.35-1.94 2.35-1.1 0-1.96-1.06-1.96-2.35zM598.44 390.94l.9-1.9 7.87 5.5-.88 1.9z" fill="#fff"/><path d="M590.55 394.58l7.87-5.5.9 1.88-7.87 5.5z" fill="#fff"/><path d="M597.97 378.98h1.8v11h-1.8z" fill="#fff"/><path d="M595.73 389.98c0-2.1 1.42-3.8 3.16-3.8 1.73 0 3.14 1.7 3.14 3.8 0 2.1-1.4 3.82-3.15 3.82s-3.17-1.7-3.17-3.82zM588.84 397c-.7-1.45-.28-3.3.92-4.13 1.2-.83 2.7-.34 3.4 1.1.7 1.45.28 3.3-.9 4.13-1.2.83-2.73.34-3.42-1.1zM605.52 398.1c-1.2-.83-1.6-2.67-.9-4.1.68-1.46 2.2-1.95 3.4-1.12 1.18.84 1.6 2.68.9 4.12-.68 1.45-2.2 1.94-3.4 1.1zM596.4 378.98c0-1.67 1.1-3.02 2.5-3.02 1.36 0 2.48 1.35 2.48 3.02 0 1.66-1.12 3.02-2.5 3.02-1.37 0-2.5-1.36-2.5-3.02z" fill="#fff"/><use xlink:href="#f" transform="matrix(1,0,0,1,575.391304347826,420.3289711850527) translate(-2.238932291666668 14.557291666666668)"/><path d="M540.17 627.5c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v79.65c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M552.67 625.5h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v1.9m0 7.6v3.78m0 7.6v3.78m0 7.6v3.78m0 7.58v3.8m0 7.58v3.8m0 7.57v3.8m0 7.58v1.9l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.6-.6-1.4v-1.9m0-7.58v-3.8m0-7.58v-3.8m0-7.6v-3.78m0-7.58v-3.8m0-7.58v-3.8m0-7.58v-3.8m0-7.57v-1.9l.6-1.4 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M585.6 679.26l-9.63-17.53c-.4-.7-.58-1.43-.58-2.23s.18-1.53.57-2.22l9.6-17.57c.38-.7.9-1.24 1.55-1.64.66-.4 1.36-.6 2.12-.6h19.26c.75 0 1.46.2 2.1.6.67.4 1.18.96 1.56 1.65l9.6 17.58c.4.7.6 1.43.6 2.22 0 .8-.2 1.54-.6 2.23l-9.6 17.57c-.38.7-.9 1.24-1.55 1.64-.64.4-1.35.6-2.1.6h-19.22c-.75-.02-1.46-.23-2.1-.64-.66-.4-1.18-.95-1.57-1.64zm0 0" fill="#0e62c2"/><path d="M604.97 653.27l-6.08-1.74-5.48 2.16-2.12 5.78 1.7 6.46 14.73 15.52h.77c.75 0 1.46-.2 2.1-.6.67-.4 1.18-.94 1.56-1.64l6.42-11.62z" fill="#0851b0"/><path d="M598.9 650.52c-2 0-3.77.65-5.32 1.96-1.55 1.3-2.54 2.97-2.98 5-.45 2.05-.23 4 .63 5.9.85 1.87 2.18 3.27 3.97 4.18 1.8.9 3.65 1.13 5.58.66 1.94-.47 3.52-1.5 4.76-3.15 1.24-1.63 1.86-3.5 1.86-5.6 0-2.46-.83-4.57-2.5-6.32-1.66-1.75-3.66-2.63-6-2.63zm0 15.76c-1.52 0-2.86-.5-4.04-1.48-1.17-1-1.93-2.25-2.26-3.8-.34-1.55-.18-3.04.48-4.47.65-1.42 1.65-2.48 3-3.17 1.37-.7 2.78-.86 4.25-.5 1.46.35 2.66 1.15 3.6 2.4.94 1.23 1.4 2.64 1.4 4.22 0 1.88-.62 3.48-1.88 4.8-1.26 1.33-2.78 2-4.56 2z" fill="#fff"/><path d="M594.85 659.14v2.78c.4.7.94 1.3 1.62 1.74v-4.52zm0 0M598.04 656.78v7.5c.54.1 1.08.1 1.62 0v-7.5zm0 0M601.38 660.3v3.32c.67-.47 1.2-1.08 1.62-1.82v-1.5zm0 0M605 665l-.87.94c-.07.08-.1.18-.1.3 0 .1.03.2.1.27l3.34 3.53c.07.08.17.12.27.12s.2-.04.28-.12l.88-.93c.07-.08.1-.17.1-.28 0-.12-.03-.22-.1-.3l-3.34-3.5c-.07-.08-.16-.13-.27-.13-.12 0-.2.04-.3.1z" fill="#fff"/><use xlink:href="#g" transform="matrix(1,0,0,1,575.3913043478261,686.5417532467532) translate(-3.2675781250000036 14.557291666666668)"/><path d="M540.17 491.6c0-1.1.9-2 2-2h113.4c1.1 0 2 .9 2 2v79.64c0 1.1-.9 2-2 2h-113.4c-1.1 0-2-.9-2-2z" fill="#fff"/><path d="M552.67 489.6h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h4.2m8.4 0h2.1l1.4.6.6 1.4v1.9m0 7.58v3.8m0 7.58v3.8m0 7.58v3.8m0 7.57v3.8m0 7.6v3.78m0 7.6v3.78m0 7.58v1.9l-.6 1.4-1.4.6h-2.1m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-4.2m-8.4 0h-2.1l-1.4-.6-.6-1.4v-1.9m0-7.58v-3.8m0-7.58v-3.8m0-7.58v-3.8m0-7.57v-3.8m0-7.58v-3.8m0-7.57v-3.8m0-7.58v-1.9l.6-1.4 1.4-.6h2.1" stroke="#000" stroke-width="2" fill="none"/><path d="M585.6 536.67l-9.63-15.4c-.4-.6-.58-1.24-.58-1.94s.18-1.35.57-1.95l9.63-15.4c.38-.6.9-1.07 1.55-1.4.65-.35 1.36-.52 2.1-.52h19.23c.75 0 1.45.18 2.1.52.66.34 1.17.82 1.56 1.4l9.63 15.4c.4.6.58 1.25.58 1.95s-.2 1.34-.58 1.94l-9.63 15.4c-.38.6-.9 1.07-1.55 1.4-.66.36-1.37.53-2.12.52h-19.26c-.75 0-1.45-.15-2.1-.5-.66-.35-1.16-.83-1.52-1.43zm0 0" fill="#0e62c2"/><path d="M604.82 509.74l-5.93 1.32-6.05-1.3 3 2.8-5.05 4.82.9.85-.86 2 1.82 1.7-1.86 1.74 4.65 4.3-2.6 1.05 10.4 9.6h5.2c.75 0 1.44-.17 2.1-.5.65-.36 1.17-.83 1.56-1.42l8.02-12.82z" fill="#0851b0"/><path d="M604.56 519.38c0-.67.6-1.22 1.32-1.22.73 0 1.32.55 1.32 1.22 0 .67-.6 1.22-1.32 1.22-.73 0-1.32-.55-1.32-1.22zM590.5 519.38c0-.67.6-1.22 1.32-1.22.73 0 1.32.55 1.32 1.22 0 .67-.6 1.22-1.32 1.22-.73 0-1.32-.55-1.32-1.22zM597.53 517.58c0-.67.6-1.2 1.32-1.2.73 0 1.32.53 1.32 1.2 0 .68-.6 1.22-1.32 1.22-.73 0-1.32-.54-1.32-1.22zM597.53 521.2c0-.67.6-1.2 1.32-1.2.73 0 1.32.53 1.32 1.2 0 .68-.6 1.22-1.32 1.22-.73 0-1.32-.54-1.32-1.2z" fill="#fff"/><path d="M600.83 511.9l4.03-2.16-1.94-1.02h-.08l-4 2.13-3.98-2.13h-.07l-1.95 1.02 4.02 2.16-6.07 3.22v2.26h2.08v-1.22l4.98-2.67v1.95h1.98v-1.96l4.98 2.66v1.22h2.05v-2.26zM604.82 522.6l-4.98 2.7v-1.96h-1.98v1.96l-4.98-2.7v-1.22h-2.08v2.26l6.07 3.25-4.02 2.12 1.94 1.05h.06l4-2.13 3.98 2.13h.08l1.94-1.05-4.03-2.13 6.04-3.26v-2.26h-2.05z" fill="#fff"/><use xlink:href="#h" transform="matrix(1,0,0,1,575.3913043478261,543.6288636363637) translate(-3.0462239583333357 14.557291666666668)"/><path d="M598.08 574.24v34.94" stroke="#000" stroke-width=".5" fill="none"/><path d="M598.33 574.24h-.5v-1h.5z"/><path d="M598.08 623.7l-4.64-14.26h9.27z" stroke="#000" stroke-width=".5"/><path d="M595.56 444.72v28.56" stroke="#000" stroke-width=".5" fill="none"/><path d="M595.8 444.72h-.5v-1h.5z"/><path d="M595.56 487.8l-4.63-14.27h9.27z" stroke="#000" stroke-width=".5"/><defs><path d="M688-137c238 0 323-102 359-325h193c-32 297-228 482-552 482-361 0-569-299-569-680 0-197 8-360 73-497 88-186 257-317 513-319 316-2 509 195 535 488h-193c-31-212-118-330-342-330-272 0-397 224-393 523v141c-3 282 115 517 376 517" id="i"/><path d="M248 11c-71 0-114-43-114-108 0-67 42-112 114-112s116 45 116 112c-1 65-45 108-116 108zm1-883c-71 0-114-43-114-108 0-67 42-112 114-112s116 45 116 112c-1 65-45 108-116 108" id="j"/><g id="a"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#i"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.678385416666668,0)" xlink:href="#j"/></g><path d="M1222-777c17 480-206 763-650 777H169v-1456h411c408 9 627 269 642 679zm-190 4c3-312-149-522-447-525H361v1141h202c308 0 473-209 469-531v-85" id="k"/><path d="M561-1102c238-4 403 126 403 351v498c0 99 13 178 38 237V0H808c-11-21-19-59-26-114C696-25 593 20 474 20c-199 0-368-130-365-320 4-251 214-359 490-356h180v-85c-1-135-86-212-229-212-115 0-232 67-233 171H131c20-205 206-316 430-320zM294-326c0 117 90 185 207 185 122 0 239-75 278-162v-222H634c-227 0-340 66-340 199" id="l"/><path d="M456 20C285 20 205-92 206-268v-671H9v-143h197v-262h185v262h202v143H391v671c-9 125 92 149 207 118V0c-49 13-96 20-142 20" id="m"/><g id="b"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#k"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#l"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#m"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#l"/></g><path d="M375 0H183v-1456h192V0" id="n"/><path d="M589-945c-131 0-219 81-264 174V0H140v-1082h175l6 136c83-104 191-156 324-156 229 0 346 129 346 387V0H806v-716c-2-153-65-229-217-229" id="o"/><path d="M521 20C240 20 96-239 96-550c0-303 147-552 427-552 132 0 235 47 309 140l9-120h169V-26c4 277-183 454-459 452-165-1-334-81-400-187l96-111c79 98 176 147 291 147 180 1 288-109 286-290v-93C750-23 649 20 521 20zm48-965c-209 0-287 178-287 416 0 217 86 392 285 392 117 0 202-53 257-159v-494c-57-103-142-155-255-155" id="p"/><path d="M599-131c141 0 220-65 285-146l113 88C906-50 770 20 589 20 281 21 93-214 93-545c0-223 93-397 233-485 74-48 154-72 240-72 300 2 449 218 445 537v77H278c0 197 129 357 321 357zm227-509c-3-180-88-310-260-310-170 0-264 140-282 310h542" id="q"/><path d="M538-131c121 0 232-49 232-156 0-50-20-88-56-117-73-60-334-92-420-143-92-55-162-110-162-238 0-190 192-317 400-317 223 0 414 129 413 338H759c0-108-110-186-227-186-119 0-215 53-215 159 0 45 18 78 53 101 76 52 331 90 416 139 98 57 169 115 169 251C955-92 760 20 538 20c-176 0-314-68-386-174-38-55-57-115-57-179h185c6 129 116 202 258 202" id="r"/><g id="c"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#n"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,3.6263020833333335,0)" xlink:href="#o"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,10.983072916666668,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,18.463541666666668,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,25.52734375,0)" xlink:href="#r"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,32.40234375,0)" xlink:href="#m"/></g><path d="M584 20C278 26 81-227 91-551c10-320 175-545 491-551 308-5 503 247 494 573-9 322-175 543-492 549zm-2-970c-208 0-305 185-305 421 0 216 106 398 307 398 211 0 307-186 307-420 0-214-109-399-309-399" id="s"/><g id="d"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#m"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,4.225260416666667,0)" xlink:href="#s"/></g><path d="M1160-420C1159-145 963 0 674 0H169v-1456h476c294-1 476 112 476 388 0 153-99 253-216 302 146 38 255 167 255 346zM678-157c171 2 289-95 289-261 0-175-95-263-286-263H361v524h317zm250-912c-2-172-104-229-283-229H361v463h290c157 3 278-84 277-234" id="t"/><path d="M341 0H156v-1082h185V0zm-91-1264c-68 0-108-42-109-105 0-62 41-107 109-107s110 44 110 107-42 105-110 105" id="u"/><path d="M1281-774c0 326-74 568-255 695l260 204-131 121L848 2C681 41 515 15 391-64 196-188 109-439 109-773c0-282 107-505 279-617 171-112 445-111 616-1 174 112 277 332 277 617zM696-143c312 0 393-279 393-632 0-307-114-536-395-536-268 0-393 232-393 523v107c-3 297 122 538 395 538" id="v"/><path d="M491 20c-241-1-355-148-355-398v-704h185v699c0 164 67 246 200 246 141 0 235-53 282-158v-787h185V0H812l-4-107C736-22 630 20 491 20" id="w"/><path d="M663-916c-163-27-288 18-338 148V0H140v-1082h180l3 125c61-97 147-145 258-145 36 0 63 5 82 14v172" id="x"/><path d="M494-271l252-811h198L509 167C425 381 299 474 84 421V271c167 16 242-37 283-173l41-110L22-1082h202" id="y"/><g id="e"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#t"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#u"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,11.536458333333334,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,19.016927083333336,0)" xlink:href="#v"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,28.18359375,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.53385416666667,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.59765625000001,0)" xlink:href="#x"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,47.22656250000001,0)" xlink:href="#y"/></g><path d="M1216-1011c-1 297-202 441-512 441H361V0H169v-1456h537c297-6 510 164 510 445zm-192 2c1-178-126-288-307-289H361v571h345c196 1 318-92 318-282" id="z"/><path d="M634-1102c292 0 422 254 422 573 0 298-143 547-420 549-137 0-242-48-317-145L310 0H140v-1536h185v573c75-93 178-139 309-139zm-44 965c207 0 281-175 281-413 0-224-79-395-283-395-122 0-210 57-263 170v468c57 113 145 170 265 170" id="A"/><path d="M177 125H18l608-1581h158" id="B"/><path d="M631-137c179 0 314-70 314-231 0-71-25-127-75-166s-141-77-272-115c-165-47-285-106-360-175s-112-154-112-255c0-245 231-397 495-397 206 0 369 89 449 218 43 68 65 142 65 223H942c0-184-129-283-321-283-172 0-299 76-301 236 0 59 26 109 76 150s135 78 256 113c205 59 345 116 436 256 35 53 50 119 50 193-2 256-231 398-507 390C333 12 90-129 80-422h193c0 187 160 285 358 285" id="C"/><g id="f"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#z"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.411458333333334,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.76171875,0)" xlink:href="#A"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,23.2421875,0)" xlink:href="#B"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,28.736979166666668,0)" xlink:href="#C"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,36.64713541666667,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.997395833333336,0)" xlink:href="#A"/></g><g id="g"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#t"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.30078125,0)" xlink:href="#u"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,11.536458333333334,0)" xlink:href="#p"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,19.016927083333336,0)" xlink:href="#v"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,28.18359375,0)" xlink:href="#w"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.53385416666667,0)" xlink:href="#q"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,42.59765625000001,0)" xlink:href="#x"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,47.22656250000001,0)" xlink:href="#y"/></g><path d="M704-1390c-164-38-297 28-288 194v114h231v143H416V0H231v-939H60v-143h171v-111c-12-274 208-416 483-347" id="D"/><path d="M341 0H156v-1536h185V0" id="E"/><path d="M1098-255l208-827h185L1176 0h-150L763-820 507 0H357L43-1082h184l213 810 252-810h149" id="F"/><g id="h"><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,0,0)" xlink:href="#k"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,8.743489583333334,0)" xlink:href="#l"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,15.99609375,0)" xlink:href="#m"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,20.3515625,0)" xlink:href="#l"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,27.604166666666668,0)" xlink:href="#D"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,32.23307291666667,0)" xlink:href="#E"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,35.46875000000001,0)" xlink:href="#s"/><use transform="matrix(0.006510416666666667,0,0,0.006510416666666667,43.07291666666667,0)" xlink:href="#F"/></g></defs></g></svg>

**Once both streaming and batch data is replicated to PubSub, the final dataflow job is responsible for ensuring data is copied to our raw BigQuery **

In [0]:
pubsub_to_bigquery= '''/content/apache-maven-3.6.0/bin/mvn compile exec:java \
-Dexec.mainClass=com.google.cloud.teleport.templates.PubSubToBigQuery \
-Dexec.args=" \
--project={project_name} \
--stagingLocation=gs://{gcs_bucket_name}/dataflow/pipelines/pubsubtobq/staging \
--tempLocation=gs://{gcs_bucket_name}/dataflow/pipelines/pubsubtobq/temp2 \
--runner=DataflowRunner \
--inputTopic=projects/{project_name}/topics/{data_source_topic_name} \
--outputTableSpec={bigquery_tableid}"'''.format(gcs_bucket_name=gcs_bucket_name,project_name=project_name,data_source_topic_name=data_source_topic_name,bigquery_tableid=project_name+":"+dataset_name+"."+table_name)

out = subprocess.run(pubsub_to_bigquery,shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out.stdout.decode().split("\n")

# Exploring Data

**The dataflow jobs that you've just launched are inserting data into BigQuery table in real-time. Execute the command below to run a SQL query to see the data being inserted. This query simply gives us the unique values for the "actor" column. You should see two unique values: batch-actor and streaming-actor. If you only see one value re-run this command again. It is likley that streaming jobs is still being consumed by dataflow and takes time before it gets inserted into the table.**

In [0]:
bq_client.query("SELECT DISTINCT actor FROM `{dataset}.{table}`".format(dataset=dataset_name,table=table_name)).to_dataframe()

# Data Transformation: Aggregation Dataflow



**Next we're going to use another dataflow job that will use the same pubsub topic but instead of inserting every record into BigQuery, the dataflow job will aggregate the incoming records on "v1_total" column. This demonstrates the ability to create a real-time aggregated column. Generally the aggregation/summary tables are created using some ETL jobs that may run in batch timeframes (i.e every 10 min) which inherently means our summary table is always stale. This job ensures data is always aggregated with the most recent data. **

**We'll start by creating a summary table in BigQuery that our dataflow job will insert aggregated results to.**

In [0]:
agg_table= table_name + "_agg"
bq_client.query('''CREATE OR REPLACE TABLE `{dataset}.{agg_table}` (user_id STRING, v1_total INT64, ts TIMESTAMP) 
  PARTITION BY DATE(ts)'''.format(dataset=dataset_name,agg_table=agg_table)).result()

In [0]:
cd /content/datagenerator/

In [0]:
 realtime_aggregation_job = '''/content/apache-maven-3.6.0/bin/mvn compile exec:java \
-Dexec.mainClass=com.google.datagenerator.RealtimeAggregationExample \
-Dexec.cleanupDaemonThreads=false \
-Dexec.args=" \
--project={project_name} \
--stagingLocation=gs://{gcs_bucket_name}/dataflow/pipelines/realtimeagg/staging \
--tempLocation=gs://{gcs_bucket_name}/dataflow/pipelines/realtimeagg/temp \
--runner=DataflowRunner \
--inputTopic=projects/{project_name}/topics/{data_source_topic_name} \
--bigQueryTable={dataset}.{agg_table}"'''.format(gcs_bucket_name=gcs_bucket_name,project_name=project_name,data_source_topic_name=data_source_topic_name,dataset=dataset_name,agg_table=agg_table)

out = subprocess.run(realtime_aggregation_job,shell=True,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out.stdout.decode().split("\n")

**Lets take a look at our summary table. Execute a few minutes after your launching the previous dataflow job.**

In [0]:
bq_client.query("SELECT * FROM `{dataset}.{table}` ORDER BY v1_total DESC LIMIT 10".format(dataset=dataset_name,table=agg_table)).to_dataframe()

# Deleting Dataflow jobs

In [0]:
%%bash -s "$project_name" 
for i in `gcloud --project $1 --format json dataflow jobs list --status active | jq  -r '.[] | .id'` ; do gcloud --project $1 dataflow jobs cancel $i ; done